In [ ]:
# v2.5
# Moved the percent area column next to left of the retime
import os
import xlrd
import pandas as pd
import re

def extract_data_from_file(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')

    # Extract data from Signal sheet
    signal_sheet = wb.sheet_by_name('Signal')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):
        # Extract Method from Sheet1 cell B6
        method = sheet1.cell_value(rowx=5, colx=1)  # B6
        
        # Extract Sequence from Sheet1 cell B5
        sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
        sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right

        # Extract File Name from Signal Sheet H2
        file_name_text = signal_sheet.cell_value(rowx=1, colx=7)  # H2
        file_name = file_name_text.rsplit("\\", 2)[-1]
        
        # Initialize dictionary for current row data
        row_data = {}

        # Add a column named 'SeqLine' with value equal to the seqline
        row_data['SeqLine'] = sheet1.cell_value(rowx=18, colx=1)  # B19
        
        # Add a column named 'Sample' with value equal to the sample name
        row_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
        
        # Add a column named 'Peak' with value equal to the current loop number n
        row_data['Peak'] = row

        # Add other data columns recursively
        for col, label in zip(range(4, 9), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)
        
        # Add Sequence, Method, and File Name to row data
        row_data['Sequence'] = sequence
        row_data['File Name'] = file_name
        row_data['Method'] = method

        # Append the row data to the list
        rows_data.append(row_data)

    if not rows_data:
        print(f"No data extracted from {file_path}")
    
    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)
    
    # Debug print to check DataFrame columns
    print(f"Columns in DataFrame for file {file_path}: {int_results_df.columns}")

    # Convert SeqLine to string for mixed digit numbers
    if 'SeqLine' in int_results_df.columns:
        int_results_df['SeqLine'] = int_results_df['SeqLine'].astype(str)
    else:
        print(f"'SeqLine' column not found in the data extracted from {file_path}")
    
    return int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []
    skipped_xls_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Check that the .xls file is an HPLC report with an int result sheet
                try:
                    # Open the Excel file
                    wb = xlrd.open_workbook(file_path)
                    wb.sheet_by_name('IntResults1')
                
                # If error, skip this file and add it to the skipped list
                except xlrd.XLRDError:
                    skipped_xls_list.append(file_path)
                    print(file_path)
                    continue
                
                # Extract data from specific cells
                int_results_df = extract_data_from_file(file_path)
                
                # Append the extracted data to the respective lists
                int_results_df_list.append(int_results_df)

    if not int_results_df_list:
        print("No '.xls' files with 'IntResults1' sheet found.")
        return

    # Concatenate all DataFrames into a single DataFrame
    int_results_df_combined = pd.concat(int_results_df_list, ignore_index=True)
    
    # Define the sorting order
    sorting_order = ['Sequence', 'SeqLine']
    
    # Convert 'SeqLine' to integer type for sorting
    int_results_df_combined['SeqLine'] = int_results_df_combined['SeqLine'].astype(str)
    int_results_df_combined['SeqLine'] = int_results_df_combined['SeqLine'].apply(extract_numeric_part)
    
    # Sort the combined DataFrame by the specified order
    int_results_df_combined.sort_values(by=sorting_order, ascending=[False, True], inplace=True)
    
    # Calculate total area for each SeqLine
    total_area_by_seqline = int_results_df_combined.groupby('SeqLine')['Area'].sum()
    
    # Calculate area percent for each peak
    int_results_df_combined['Area_Percent'] = int_results_df_combined.apply(
        lambda row: (row['Area'] / total_area_by_seqline[row['SeqLine']]) * 100, axis=1
    )
    
    # Reorder columns to move 'Area_Percent' to the left of 'RetTime'
    cols = ['Sequence', 'SeqLine', 'Sample', 'Peak', 'Area_Percent', 'RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'File Name', 'Method']
    int_results_df_combined = int_results_df_combined[cols]
    
    # Print the extracted DataFrame
    print("\nIntResults DataFrame:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    file_name = "HPLC_Summary_" + parent_directory.rsplit("\\", 2)[-1] + ".csv"
    try:
        if int_results_df_combined['Sequence'].iloc[0] == int_results_df_combined['Sequence'].iloc[-1]:
            int_results_csv_path = os.path.join(parent_directory, file_name)
            int_results_df_combined.to_csv(int_results_csv_path, index=False)
    except:
        int_results_csv_path = os.path.join(parent_directory, "HPLC_Summary.csv")
        int_results_df_combined.to_csv(int_results_csv_path, index=False)
    
    # Print skipped file list
    print("Skipped files:", skipped_xls_list)
    
    # Print location of output file
    print(f"Results saved to: {int_results_csv_path}")
    
    # Input pause so user can see result path
    input("Press the <ENTER> key to continue...")

        
# Define a function to extract the numeric part from the SeqLine string
def extract_numeric_part(seqline):
    # Use regular expression to find the numeric part
    match = re.search(r'\d+', seqline)
    if match:
        return int(match.group())  # Return the numeric part as integer
    else:
        return 0  # If no numeric part found, return 0

if __name__ == "__main__":
    main()


Enter the parent directory path: C:\Users\JoelTencer\Documents\HPLC-1\SEC\2024 07 21 ATTO-005 HEAT STRESS TEST 2024-07-21 13-17-44
WARNING *** file size (31937) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31937) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Columns in DataFrame for file C:\Users\JoelTencer\Documents\HPLC-1\SEC\2024 07 21 ATTO-005 HEAT STRESS TEST 2024-07-21 13-17-44\003-0101.D\REPORT01.xls: Index(['SeqLine', 'Sample', 'Peak', 'RetTime', 'Area', 'Height', 'Width',
       'Symmetry', 'Sequence', 'File Name', 'Method'],
      dtype='object')
WARNING *** file size (33623) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (33623) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is no

WARNING *** file size (34468) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (34468) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Columns in DataFrame for file C:\Users\JoelTencer\Documents\HPLC-1\SEC\2024 07 21 ATTO-005 HEAT STRESS TEST 2024-07-21 13-17-44\1BI-2101.D\REPORT01.xls: Index(['SeqLine', 'Sample', 'Peak', 'RetTime', 'Area', 'Height', 'Width',
       'Symmetry', 'Sequence', 'File Name', 'Method'],
      dtype='object')
WARNING *** file size (34468) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (34468) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Columns in DataFrame for file C:\Users\JoelTencer\Documents\HPLC-1\SEC\2024 07 21 ATTO-005 HEAT STRESS TEST 2024-07-21 13-17

In [1]:
#v 2.4
# Manually calculates the area percent by adding the areas and dividing the area by the sum.

import os
import xlrd
import pandas as pd
import re

def extract_data_from_file(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')

    # Extract data from Signal sheet
    signal_sheet = wb.sheet_by_name('Signal')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):
        # Extract Method from Sheet1 cell B6
        method = sheet1.cell_value(rowx=5, colx=1)  # B6
        
        # Extract Sequence from Sheet1 cell B5
        sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
        sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right

        # Extract File Name from Signal Sheet H2
        file_name_text = signal_sheet.cell_value(rowx=1, colx=7)  # H2
        file_name = file_name_text.rsplit("\\", 2)[-1]
        
        # Initialize dictionary for current row data
        row_data = {}

        # Add a column named 'SeqLine' with value equal to the seqline
        row_data['SeqLine'] = sheet1.cell_value(rowx=18, colx=1)  # B19
        
        # Add a column named 'Sample' with value equal to the sample name
        row_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
        
        # Add a column named 'Peak' with value equal to the current loop number n
        row_data['Peak'] = row

        # Add other data columns recursively
        for col, label in zip(range(4, 9), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)
        
        # Add Sequence, Method, and File Name to row data
        row_data['Sequence'] = sequence
        row_data['File Name'] = file_name
        row_data['Method'] = method

        # Append the row data to the list
        rows_data.append(row_data)

    if not rows_data:
        print(f"No data extracted from {file_path}")
    
    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)
    
    # Debug print to check DataFrame columns
    print(f"Columns in DataFrame for file {file_path}: {int_results_df.columns}")

    # Convert SeqLine to string for mixed digit numbers
    if 'SeqLine' in int_results_df.columns:
        int_results_df['SeqLine'] = int_results_df['SeqLine'].astype(str)
    else:
        print(f"'SeqLine' column not found in the data extracted from {file_path}")
    
    return int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []
    skipped_xls_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Check that the .xls file is an HPLC report with an int result sheet
                try:
                    # Open the Excel file
                    wb = xlrd.open_workbook(file_path)
                    wb.sheet_by_name('IntResults1')
                
                # If error, skip this file and add it to the skipped list
                except xlrd.XLRDError:
                    skipped_xls_list.append(file_path)
                    print(file_path)
                    continue
                
                # Extract data from specific cells
                int_results_df = extract_data_from_file(file_path)
                
                # Append the extracted data to the respective lists
                int_results_df_list.append(int_results_df)

    if not int_results_df_list:
        print("No '.xls' files with 'IntResults1' sheet found.")
        return

    # Concatenate all DataFrames into a single DataFrame
    int_results_df_combined = pd.concat(int_results_df_list, ignore_index=True)
    
    # Define the sorting order
    sorting_order = ['Sequence', 'SeqLine']
    
    # Convert 'SeqLine' to integer type for sorting
    int_results_df_combined['SeqLine'] = int_results_df_combined['SeqLine'].astype(str)
    int_results_df_combined['SeqLine'] = int_results_df_combined['SeqLine'].apply(extract_numeric_part)
    
    # Sort the combined DataFrame by the specified order
    int_results_df_combined.sort_values(by=sorting_order, ascending=[False, True], inplace=True)
    
    # Calculate total area for each SeqLine
    total_area_by_seqline = int_results_df_combined.groupby('SeqLine')['Area'].sum()
    
    # Calculate area percent for each peak
    int_results_df_combined['Area_Percent'] = int_results_df_combined.apply(
        lambda row: (row['Area'] / total_area_by_seqline[row['SeqLine']]) * 100, axis=1
    )
    
    # Print the extracted DataFrame
    print("\nIntResults DataFrame:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    file_name = "HPLC_Summary_" + parent_directory.rsplit("\\", 2)[-1] + ".csv"
    try:
        if int_results_df_combined['Sequence'].iloc[0] == int_results_df_combined['Sequence'].iloc[-1]:
            int_results_csv_path = os.path.join(parent_directory, file_name)
            int_results_df_combined.to_csv(int_results_csv_path, index=False)
    except:
        int_results_csv_path = os.path.join(parent_directory, "HPLC_Summary.csv")
        int_results_df_combined.to_csv(int_results_csv_path, index=False)
    
    # Print skipped file list
    print("Skipped files:", skipped_xls_list)
    
    # Print location of output file
    print(f"Results saved to: {int_results_csv_path}")
    
    # Input pause so user can see result path
    input("Press the <ENTER> key to continue...")

        
# Define a function to extract the numeric part from the SeqLine string
def extract_numeric_part(seqline):
    # Use regular expression to find the numeric part
    match = re.search(r'\d+', seqline)
    if match:
        return int(match.group())  # Return the numeric part as integer
    else:
        return 0  # If no numeric part found, return 0

if __name__ == "__main__":
    main()


Enter the parent directory path: C:\Users\JoelTencer\Documents\HPLC-1\SEC\2024 07 21 ATTO-005 HEAT STRESS TEST 2024-07-21 13-17-44
WARNING *** file size (31937) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31937) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Columns in DataFrame for file C:\Users\JoelTencer\Documents\HPLC-1\SEC\2024 07 21 ATTO-005 HEAT STRESS TEST 2024-07-21 13-17-44\003-0101.D\REPORT01.xls: Index(['SeqLine', 'Sample', 'Peak', 'RetTime', 'Area', 'Height', 'Width',
       'Symmetry', 'Sequence', 'File Name', 'Method'],
      dtype='object')
WARNING *** file size (33623) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (33623) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is no

WARNING *** file size (32782) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32782) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Columns in DataFrame for file C:\Users\JoelTencer\Documents\HPLC-1\SEC\2024 07 21 ATTO-005 HEAT STRESS TEST 2024-07-21 13-17-44\1BD-1601.D\REPORT01.xls: Index(['SeqLine', 'Sample', 'Peak', 'RetTime', 'Area', 'Height', 'Width',
       'Symmetry', 'Sequence', 'File Name', 'Method'],
      dtype='object')
WARNING *** file size (31939) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31939) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Columns in DataFrame for file C:\Users\JoelTencer\Documents\HPLC-1\SEC\2024 07 21 ATTO-005 HEAT STRESS TEST 2024-07-21 13-17

Press the <ENTER> key to continue...


In [ ]:
# v2.3
# fixed seqline error with chatgpt
# Added a check to see if rows_data is empty after the data extraction loop.
# Added debug prints to check the columns in the DataFrame after creating it.
# Added a check to ensure 'SeqLine' exists in the DataFrame before attempting to convert it to a string type.

import os
import xlrd
import pandas as pd
import re

def extract_data_from_file(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')

    # Extract data from Signal sheet
    signal_sheet = wb.sheet_by_name('Signal')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):
        # Extract Method from Sheet1 cell B6
        method = sheet1.cell_value(rowx=5, colx=1)  # B6
        
        # Extract Sequence from Sheet1 cell B5
        sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
        sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right

        # Extract File Name from Signal Sheet H2
        file_name_text = signal_sheet.cell_value(rowx=1, colx=7)  # H2
        file_name = file_name_text.rsplit("\\", 2)[-1]
        
        # Initialize dictionary for current row data
        row_data = {}

        # Add a column named 'SeqLine' with value equal to the seqline
        row_data['SeqLine'] = sheet1.cell_value(rowx=18, colx=1)  # B19
        
        # Add a column named 'Sample' with value equal to the sample name
        row_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
        
        # Add a column named 'Peak' with value equal to the current loop number n
        row_data['Peak'] = row

        # Check if column 22 has a value, set area percent accordingly
        try:
            area_percent = int_results_sheet.cell_value(rowx=row, colx=22)
        except IndexError:
            area_percent = ""
        row_data['Area_Percent'] = area_percent
        
        # Add other data columns recursively
        for col, label in zip(range(4, 9), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)

        
        # Add Sequence, Method, and File Name to row data
        row_data['Sequence'] = sequence
        row_data['File Name'] = file_name
        row_data['Method'] = method

        # Append the row data to the list
        rows_data.append(row_data)

    if not rows_data:
        print(f"No data extracted from {file_path}")
    
    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)
    
    # Debug print to check DataFrame columns
    print(f"Columns in DataFrame for file {file_path}: {int_results_df.columns}")

    # Convert SeqLine to string for mixed digit numbers
    if 'SeqLine' in int_results_df.columns:
        int_results_df['SeqLine'] = int_results_df['SeqLine'].astype(str)
    else:
        print(f"'SeqLine' column not found in the data extracted from {file_path}")
    
    return int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []
    skipped_xls_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Check that the .xls file is an HPLC report with an int result sheet
                try:
                    # Open the Excel file
                    wb = xlrd.open_workbook(file_path)
                    wb.sheet_by_name('IntResults1')
                
                # If error, skip this file and add it to the skipped list
                except xlrd.XLRDError:
                    skipped_xls_list.append(file_path)
                    print(file_path)
                    continue
                
                # Extract data from specific cells
                int_results_df = extract_data_from_file(file_path)
                
                # Append the extracted data to the respective lists
                int_results_df_list.append(int_results_df)

    if not int_results_df_list:
        print("No '.xls' files with 'IntResults1' sheet found.")
        return

    # Concatenate all DataFrames into a single DataFrame
    int_results_df_combined = pd.concat(int_results_df_list, ignore_index=True)
    
    # Define the sorting order
    sorting_order = ['Sequence', 'SeqLine']
    
    # Convert 'SeqLine' to integer type for sorting
    int_results_df_combined['SeqLine'] = int_results_df_combined['SeqLine'].astype(str)
    int_results_df_combined['SeqLine'] = int_results_df_combined['SeqLine'].apply(extract_numeric_part)
    
    # Sort the combined DataFrame by the specified order
    int_results_df_combined.sort_values(by=sorting_order, ascending=[False, True], inplace=True)
    
    # Print the extracted DataFrame
    # print("\nIntResults DataFrame:")
    # print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    file_name = "HPLC_Summary_" + parent_directory.rsplit("\\", 2)[-1] + ".csv"
    try:
        if int_results_df_combined['Sequence'].iloc[0] == int_results_df_combined['Sequence'].iloc[-1]:
            int_results_csv_path = os.path.join(parent_directory, file_name)
            int_results_df_combined.to_csv(int_results_csv_path, index=False)
    except:
        int_results_csv_path = os.path.join(parent_directory, "HPLC_Summary.csv")
        int_results_df_combined.to_csv(int_results_csv_path, index=False)
    
    # Print skipped file list
    print("Skipped files:", skipped_xls_list)
    
    # Print location of output file
    print(f"Results saved to: {int_results_csv_path}")
    
    # Input pause so user can see result path
    input("Press the <ENTER> key to continue...")

        
# Define a function to extract the numeric part from the SeqLine string
def extract_numeric_part(seqline):
    # Use regular expression to find the numeric part
    match = re.search(r'\d+', seqline)
    if match:
        return int(match.group())  # Return the numeric part as integer
    else:
        return 0  # If no numeric part found, return 0

if __name__ == "__main__":
    main()


Enter the parent directory path: \\10.2.20.249\Lab Data\Joel Tencer\Agilent HPLC 2\SEC\2024 06 18 TL1A ATTO AND NANO FC 2024-06-18 13-37-10
WARNING *** file size (43135) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (43135) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Columns in DataFrame for file \\10.2.20.249\Lab Data\Joel Tencer\Agilent HPLC 2\SEC\2024 06 18 TL1A ATTO AND NANO FC 2024-06-18 13-37-10\003-0101.D\REPORT01.xls: Index(['SeqLine', 'Sample', 'Peak', 'Area_Percent', 'RetTime', 'Area',
       'Height', 'Width', 'Symmetry', 'Sequence', 'File Name', 'Method'],
      dtype='object')
WARNING *** file size (38810) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (38810) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: 

Columns in DataFrame for file \\10.2.20.249\Lab Data\Joel Tencer\Agilent HPLC 2\SEC\2024 06 18 TL1A ATTO AND NANO FC 2024-06-18 13-37-10\1BB-1901.D\REPORT01.xls: Index(['SeqLine', 'Sample', 'Peak', 'Area_Percent', 'RetTime', 'Area',
       'Height', 'Width', 'Symmetry', 'Sequence', 'File Name', 'Method'],
      dtype='object')
WARNING *** file size (38828) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (38828) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Columns in DataFrame for file \\10.2.20.249\Lab Data\Joel Tencer\Agilent HPLC 2\SEC\2024 06 18 TL1A ATTO AND NANO FC 2024-06-18 13-37-10\1BC-2501.D\REPORT01.xls: Index(['SeqLine', 'Sample', 'Peak', 'Area_Percent', 'RetTime', 'Area',
       'Height', 'Width', 'Symmetry', 'Sequence', 'File Name', 'Method'],
      dtype='object')
WARNING *** file size (35688) not 512 + multiple of sect

WARNING *** file size (37993) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Columns in DataFrame for file \\10.2.20.249\Lab Data\Joel Tencer\Agilent HPLC 2\SEC\2024 06 18 TL1A ATTO AND NANO FC 2024-06-18 13-37-10\1CH-5601.D\REPORT01.xls: Index(['SeqLine', 'Sample', 'Peak', 'Area_Percent', 'RetTime', 'Area',
       'Height', 'Width', 'Symmetry', 'Sequence', 'File Name', 'Method'],
      dtype='object')
WARNING *** file size (39020) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (39020) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Columns in DataFrame for file \\10.2.20.249\Lab Data\Joel Tencer\Agilent HPLC 2\SEC\2024 06 18 TL1A ATTO AND NANO FC 2024-06-18 13-37-10\1DA-1401.D\REPORT01.xls: Index(['SeqLine', 'Sample', 'Peak', 'Area_Percent', 'RetTime', 'Area',
       'H

WARNING *** file size (37150) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (37150) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Columns in DataFrame for file \\10.2.20.249\Lab Data\Joel Tencer\Agilent HPLC 2\SEC\2024 06 18 TL1A ATTO AND NANO FC 2024-06-18 13-37-10\1EH-5801.D\REPORT01.xls: Index(['SeqLine', 'Sample', 'Peak', 'Area_Percent', 'RetTime', 'Area',
       'Height', 'Width', 'Symmetry', 'Sequence', 'File Name', 'Method'],
      dtype='object')
WARNING *** file size (39022) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (39022) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Columns in DataFrame for file \\10.2.20.249\Lab Data\Joel Tencer\Agilent HPLC 2\SEC\2024 06 18 TL1A

WARNING *** file size (37346) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (37346) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Columns in DataFrame for file \\10.2.20.249\Lab Data\Joel Tencer\Agilent HPLC 2\SEC\2024 06 18 TL1A ATTO AND NANO FC 2024-06-18 13-37-10\1GG-5201.D\REPORT01.xls: Index(['SeqLine', 'Sample', 'Peak', 'Area_Percent', 'RetTime', 'Area',
       'Height', 'Width', 'Symmetry', 'Sequence', 'File Name', 'Method'],
      dtype='object')
WARNING *** file size (37993) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (37993) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Columns in DataFrame for file \\10.2.20.249\Lab Data\Joel Tencer\Agilent HPLC 2\SEC\2024 06 18 TL1A

In [2]:
# v2.2
# Added check to main loop to skip if the xls file does not contain the intresult sheet
# Added report at end which files were skipped for not containing this
# Add DAD signal for each?
# Must use short or detail report, full does not have area percent
# Test with other column run data

import os
import xlrd
import pandas as pd
import re

def extract_data_from_file(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')

    # Extract data from Signal sheet
    signal_sheet = wb.sheet_by_name('Signal')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):
        # Extract Method from Sheet1 cell B6
        method = sheet1.cell_value(rowx=5, colx=1)  # B6
        
        # Extract Sequence from Sheet1 cell B5
        sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
        sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right

        # Extract File Name from Signal Sheet H2
        file_name_text = signal_sheet.cell_value(rowx=1, colx=7)  # H2
        file_name = file_name_text.rsplit("\\", 2)[-1]
        
        # Initialize dictionary for current row data
        row_data = {}

        # Add a column named 'SeqLine' with value equal to the sample name
        row_data['SeqLine'] = sheet1.cell_value(rowx=18, colx=1)  # B19
        
        # Add a column named 'Sample' with value equal to the sample name
        row_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
        
        # Add a column named 'Peak' with value equal to the current loop number n
        row_data['Peak'] = row

        # Check if column 22 has a value, set area percent accordingly
        try:
            area_percent = int_results_sheet.cell_value(rowx=row, colx=22)
        except IndexError:
            area_percent = ""
        row_data['Area_Percent'] = area_percent
        
        # Add other data columns recursively
        for col, label in zip(range(4, 9), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)

        
        # Add Sequence, Method, and File Name to row data
        row_data['Sequence'] = sequence
        row_data['File Name'] = file_name
        row_data['Method'] = method

        # Append the row data to the list
        rows_data.append(row_data)

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    # Convert SeqLine to string for mixed digit numbers
    int_results_df['SeqLine'] = int_results_df['SeqLine'].astype(str)

    return int_results_df


def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []
    skipped_xls_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Check that the .xls file is an HPLC report with an int result sheet
                try:
                    # Open the Excel file
                    wb = xlrd.open_workbook(file_path)
                    wb.sheet_by_name('IntResults1')
                
                # If error, skip this file and add it to the skipped list
                except xlrd.XLRDError:
                    skipped_xls_list.append(file_path)
                    print(file_path)
                    continue
                
                # Extract data from specific cells
                int_results_df = extract_data_from_file(file_path)
                
                # Append the extracted data to the respective lists
                int_results_df_list.append(int_results_df)

    if not int_results_df_list:
        print("No '.xls' files with 'IntResults1' sheet found.")
        return

    # Concatenate all DataFrames into a single DataFrame
    int_results_df_combined = pd.concat(int_results_df_list, ignore_index=True)
    
    # Define the sorting order
    sorting_order = ['Sequence', 'SeqLine']
    
    # Convert 'SeqLine' to integer type for sorting
    int_results_df_combined['SeqLine'] = int_results_df_combined['SeqLine'].astype(str)
    int_results_df_combined['SeqLine'] = int_results_df_combined['SeqLine'].apply(extract_numeric_part)
    
    # Sort the combined DataFrame by the specified order
    int_results_df_combined.sort_values(by=sorting_order, ascending=[False, True], inplace=True)
    
    # Print the extracted DataFrame
    # print("\nIntResults DataFrame:")
    # print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    file_name = "HPLC_Summary_" + parent_directory.rsplit("\\", 2)[-1] + ".csv"
    try:
        if int_results_df_combined['Sequence'].iloc[0] == int_results_df_combined['Sequence'].iloc[-1]:
            int_results_csv_path = os.path.join(parent_directory, file_name)
            int_results_df_combined.to_csv(int_results_csv_path, index=False)
    except:
        int_results_csv_path = os.path.join(parent_directory, "HPLC_Summary.csv")
        int_results_df_combined.to_csv(int_results_csv_path, index=False)
    
    # Print skipped file list
    print("Skipped files:", skipped_xls_list)
    
    # Print location of output file
    print(f"Results saved to: {int_results_csv_path}")
    
    # Input pause so user can see result path
    input("Press the <ENTER> key to continue...")

        
# Define a function to extract the numeric part from the SeqLine string
def extract_numeric_part(seqline):
    # Use regular expression to find the numeric part
    match = re.search(r'\d+', seqline)
    if match:
        return int(match.group())  # Return the numeric part as integer
    else:
        return 0  # If no numeric part found, return 0

if __name__ == "__main__":
    main()

Enter the parent directory path: \\10.2.20.249\Lab Data\Joel Tencer\Agilent HPLC 2\SEC\2024 06 18 TL1A ATTO AND NANO FC 2024-06-18 13-37-10
WARNING *** file size (43135) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (43135) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (38810) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (38810) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (34979) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (34979) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WA

KeyError: 'SeqLine'

In [2]:
# v2.1
# Added if multiple sequences are scraped to just name the csv the default name
# Added pause at end so user knows where file is saved
# Add DAD signal for each?
# Must use short or detail report, full does not have area percent
# Test with other column run data

import os
import xlrd
import pandas as pd
import re

def extract_data_from_file(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')

    # Extract data from Signal sheet
    signal_sheet = wb.sheet_by_name('Signal')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):
        # Extract Method from Sheet1 cell B6
        method = sheet1.cell_value(rowx=5, colx=1)  # B6
        
        # Extract Sequence from Sheet1 cell B5
        sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
        sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right

        # Extract File Name from Signal Sheet H2
        file_name_text = signal_sheet.cell_value(rowx=1, colx=7)  # H2
        file_name = file_name_text.rsplit("\\", 2)[-1]
        
        # Initialize dictionary for current row data
        row_data = {}

        # Add a column named 'SeqLine' with value equal to the sample name
        row_data['SeqLine'] = sheet1.cell_value(rowx=18, colx=1)  # B19
        
        # Add a column named 'Sample' with value equal to the sample name
        row_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
        
        # Add a column named 'Peak' with value equal to the current loop number n
        row_data['Peak'] = row

        # Check if column 22 has a value, set area percent accordingly
        try:
            area_percent = int_results_sheet.cell_value(rowx=row, colx=22)
        except IndexError:
            area_percent = ""
        row_data['Area_Percent'] = area_percent
        
        # Add other data columns recursively
        for col, label in zip(range(4, 9), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)

        
        # Add Sequence, Method, and File Name to row data
        row_data['Sequence'] = sequence
        row_data['File Name'] = file_name
        row_data['Method'] = method

        # Append the row data to the list
        rows_data.append(row_data)

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    # Convert SeqLine to string for mixed digit numbers
    int_results_df['SeqLine'] = int_results_df['SeqLine'].astype(str)

    return int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                int_results_df = extract_data_from_file(file_path)
                
                # Append the extracted data to the respective lists
                int_results_df_list.append(int_results_df)

    # Concatenate all DataFrames into a single DataFrame
    int_results_df_combined = pd.concat(int_results_df_list, ignore_index=True)
    
    # Define the sorting order
    sorting_order = ['Sequence', 'SeqLine']
    
    # Convert 'SeqLine' to integer type for sorting
    int_results_df_combined['SeqLine'] = int_results_df_combined['SeqLine'].astype(str)
    int_results_df_combined['SeqLine'] = int_results_df_combined['SeqLine'].apply(extract_numeric_part)
    
    # Sort the combined DataFrame by the specified order
    int_results_df_combined.sort_values(by=sorting_order, ascending=[False, True], inplace=True)
    
    # Print the extracted DataFrame
    # print("\nIntResults DataFrame:")
    # print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    file_name = "HPLC_Summary_" + parent_directory.rsplit("\\", 2)[-1] + ".csv"
    try:
        if int_results_df_combined['Sequence'].iloc[0] == int_results_df_combined['Sequence'].iloc[-1]:
            int_results_csv_path = os.path.join(parent_directory, file_name)
            int_results_df_combined.to_csv(int_results_csv_path, index=False)
    except:
        int_results_csv_path = os.path.join(parent_directory, "HPLC_Summary.csv")
        int_results_df_combined.to_csv(int_results_csv_path, index=False)
    
    #Print location of output file:
    print(f"Results saved to: {int_results_csv_path}")
    
    #Input pause so user can see result path:
    input("Press the <ENTER> key to continue...")
        
# Define a function to extract the numeric part from the SeqLine string
def extract_numeric_part(seqline):
    # Use regular expression to find the numeric part
    match = re.search(r'\d+', seqline)
    if match:
        return int(match.group())  # Return the numeric part as integer
    else:
        return 0  # If no numeric part found, return 0

if __name__ == "__main__":
    main()

Enter the parent directory path: C:\Users\JoelTencer\OneDrive - Attovia\Weekly Schedule


ValueError: No objects to concatenate

In [18]:
# v2
# Added first sort by sequence name, second seqLine
# Add DAD signal for each?
# Must use short or detail report, full does not have area percent
# Test with other column run data

import os
import xlrd
import pandas as pd
import re

def extract_data_from_file(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')

    # Extract data from Signal sheet
    signal_sheet = wb.sheet_by_name('Signal')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):
        # Extract Method from Sheet1 cell B6
        method = sheet1.cell_value(rowx=5, colx=1)  # B6
        
        # Extract Sequence from Sheet1 cell B5
        sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
        sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right

        # Extract File Name from Signal Sheet H2
        file_name_text = signal_sheet.cell_value(rowx=1, colx=7)  # H2
        file_name = file_name_text.rsplit("\\", 2)[-1]
        
        # Initialize dictionary for current row data
        row_data = {}

        # Add a column named 'SeqLine' with value equal to the sample name
        row_data['SeqLine'] = sheet1.cell_value(rowx=18, colx=1)  # B19
        
        # Add a column named 'Sample' with value equal to the sample name
        row_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
        
        # Add a column named 'Peak' with value equal to the current loop number n
        row_data['Peak'] = row

        # Check if column 22 has a value, set area percent accordingly
        try:
            area_percent = int_results_sheet.cell_value(rowx=row, colx=22)
        except IndexError:
            area_percent = ""
        row_data['Area_Percent'] = area_percent
        
        # Add other data columns recursively
        for col, label in zip(range(4, 9), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)

        
        # Add Sequence, Method, and File Name to row data
        row_data['Sequence'] = sequence
        row_data['File Name'] = file_name
        row_data['Method'] = method

        # Append the row data to the list
        rows_data.append(row_data)

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    # Convert SeqLine to string for mixed digit numbers
    int_results_df['SeqLine'] = int_results_df['SeqLine'].astype(str)

    return int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                int_results_df = extract_data_from_file(file_path)
                
                # Append the extracted data to the respective lists
                int_results_df_list.append(int_results_df)

    # Concatenate all DataFrames into a single DataFrame
    int_results_df_combined = pd.concat(int_results_df_list, ignore_index=True)
    
    # Define the sorting order
    sorting_order = ['Sequence', 'SeqLine']
    
    # Convert 'SeqLine' to integer type for sorting
    int_results_df_combined['SeqLine'] = int_results_df_combined['SeqLine'].astype(str)
    int_results_df_combined['SeqLine'] = int_results_df_combined['SeqLine'].apply(extract_numeric_part)
    
    # Sort the combined DataFrame by the specified order
    int_results_df_combined.sort_values(by=sorting_order, ascending=[False, True], inplace=True)
    
    # Print the extracted DataFrame
    # print("\nIntResults DataFrame:")
    # print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    file_name = "HPLC_Summary_" + parent_directory.rsplit("\\", 2)[-1] + ".csv"
    try:
        int_results_csv_path = os.path.join(parent_directory, file_name)
        int_results_df_combined.to_csv(int_results_csv_path, index=False)
    except:
        int_results_csv_path = os.path.join(parent_directory, "HPLC_Summary.csv")
        int_results_df_combined.to_csv(int_results_csv_path, index=False)
    
    print(f"Results saved to: {int_results_csv_path}")
    
# Define a function to extract the numeric part from the SeqLine string
def extract_numeric_part(seqline):
    # Use regular expression to find the numeric part
    match = re.search(r'\d+', seqline)
    if match:
        return int(match.group())  # Return the numeric part as integer
    else:
        return 0  # If no numeric part found, return 0

if __name__ == "__main__":
    main()

Enter the parent directory path: C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 05 06 ASYM IL13_HZ_4C12_2BB3_5D4 2024-05-06 14-29-28.zip\2024 05 06 ASYM IL13_HZ_4C12_2BB3_5D4 2024-05-06 14-29-28


ValueError: No objects to concatenate

In [19]:
# v1.9
# Added SeqLine as first item then sort by seqLine
# Add DAD signal for each?
# Must use short or detail report, full does not have area percent
# Test with other column run data

import os
import xlrd
import pandas as pd
import re

def extract_data_from_file(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')

    # Extract data from Signal sheet
    signal_sheet = wb.sheet_by_name('Signal')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):
        # Extract Method from Sheet1 cell B6
        method = sheet1.cell_value(rowx=5, colx=1)  # B6
        
        # Extract Sequence from Sheet1 cell B5
        sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
        sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right

        # Extract File Name from Signal Sheet H2
        file_name_text = signal_sheet.cell_value(rowx=1, colx=7)  # H2
        file_name = file_name_text.rsplit("\\", 2)[-1]
        
        # Initialize dictionary for current row data
        row_data = {}

        # Add a column named 'SeqLine' with value equal to the sample name
        row_data['SeqLine'] = sheet1.cell_value(rowx=18, colx=1)  # B19
        
        # Add a column named 'Sample' with value equal to the sample name
        row_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
        
        # Add a column named 'Peak' with value equal to the current loop number n
        row_data['Peak'] = row

        # Check if column 22 has a value, set area percent accordingly
        try:
            area_percent = int_results_sheet.cell_value(rowx=row, colx=22)
        except IndexError:
            area_percent = ""
        row_data['Area_Percent'] = area_percent
        
        # Add other data columns recursively
        for col, label in zip(range(4, 9), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)

        
        # Add Sequence, Method, and File Name to row data
        row_data['Sequence'] = sequence
        row_data['File Name'] = file_name
        row_data['Method'] = method

        # Append the row data to the list
        rows_data.append(row_data)

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    # Convert SeqLine to string for mixed digit numbers
    int_results_df['SeqLine'] = int_results_df['SeqLine'].astype(str)

    return int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                int_results_df = extract_data_from_file(file_path)
                
                # Append the extracted data to the respective lists
                int_results_df_list.append(int_results_df)

    # Concatenate all DataFrames into a single DataFrame
    int_results_df_combined = pd.concat(int_results_df_list, ignore_index=True)
    
    # Sort the combined DataFrame by SeqLine
    int_results_df_combined['SeqLine'] = int_results_df_combined['SeqLine'].astype(str)
    int_results_df_combined['SeqLine'] = int_results_df_combined['SeqLine'].apply(extract_numeric_part)
    int_results_df_combined = int_results_df_combined.sort_values(by='SeqLine')
    
    # Print the extracted DataFrame
    # print("\nIntResults DataFrame:")
    # print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    file_name = "HPLC_Summary_" + parent_directory.rsplit("\\", 2)[-1] + ".csv"
    try:
        int_results_csv_path = os.path.join(parent_directory, file_name)
        int_results_df_combined.to_csv(int_results_csv_path, index=False)
    except:
        int_results_csv_path = os.path.join(parent_directory, "HPLC_Summary.csv")
        int_results_df_combined.to_csv(int_results_csv_path, index=False)
    
    print(f"Results saved to: {int_results_csv_path}")

# Define a function to extract the numeric part from the SeqLine string
def extract_numeric_part(seqline):
    # Use regular expression to find the numeric part
    match = re.search(r'\d+', seqline)
    if match:
        return int(match.group())  # Return the numeric part as integer
    else:
        return 0  # If no numeric part found, return 0

if __name__ == "__main__":
    main()


Enter the parent directory path: C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 05 06 ASYM IL13_HZ_4C12_2BB3_5D4 2024-05-06 14-29-28.zip\2024 05 06 ASYM IL13_HZ_4C12_2BB3_5D4 2024-05-06 14-29-28


ValueError: No objects to concatenate

In [17]:
# v1.8
# Added parent folder to file name of report, if error just call HPLC Summary Report
# changed where sequence name is taken from Sheet1 B15 to Sheet1 B5
# Add DAD signal for each?
# Look into full report save options, why was fileinfo1 sheet in one thing and not another, check all are the same
# Test with other column run data

import os
import xlrd
import pandas as pd

def extract_data_from_file(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')

    #Extract data from FileInfo1 sheet
    #fileinfo1_sheet = wb.sheet_by_name('FileInfo1')

    # Extract data from Signal sheet
    signal_sheet = wb.sheet_by_name('Signal')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):
        # Extract Method from Sheet1 cell B6
        method = sheet1.cell_value(rowx=5, colx=1)  # B6
        
        # Extract Sequence from Sheet1 cell B5
        sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
        sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right

        # Extract file name from FileInfo1 cell D2
        #file_name = fileinfo1_sheet.cell_value(rowx=1, colx=3)  # D2

        # Extract File Name from Signal Sheet H2
        file_name_text = signal_sheet.cell_value(rowx=1, colx=7)  # H2
        file_name = file_name_text.rsplit("\\", 2)[-1]
        
        
        # Initialize dictionary for current row data
        row_data = {}

        # Add a column named 'Sample' with value equal to the sample name
        row_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
        
        # Add a column named 'Peak' with value equal to the current loop number n
        row_data['Peak'] = row

        # Check if column 22 has a value, set area percent accordingly
        try:
            area_percent = int_results_sheet.cell_value(rowx=row, colx=22)
        except IndexError:
            area_percent = ""
        row_data['Area_Percent'] = area_percent
        
        # Add other data columns recursively
        for col, label in zip(range(4, 9), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)

        
        # Add Sequence, Method, and File Name to row data
        row_data['Sequence'] = sequence
        row_data['File Name'] = file_name
        row_data['Method'] = method

        # Append the row data to the list
        rows_data.append(row_data)

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)
   # print('First loop: ', int_results_df)

    return int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                int_results_df = extract_data_from_file(file_path)
                
                # Append the extracted data to the respective lists
                int_results_df_list.append(int_results_df)
                #print('Main Loop: ', int_results_df)

    # Concatenate all DataFrames into a single DataFrame
    int_results_df_combined = pd.concat(int_results_df_list, ignore_index=True)
    
    # Print the extracted DataFrame
   # print("\nIntResults Dataframe:")
   # print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    file_name = "HPLC_Summary_" + parent_directory.rsplit("\\", 2)[-1] + ".csv"
    try:
        int_results_csv_path = os.path.join(parent_directory, file_name)
        int_results_df_combined.to_csv(int_results_csv_path, index=False)
    except:
        int_results_csv_path = os.path.join(parent_directory, "HPLC_Summary.csv")
        int_results_df_combined.to_csv(int_results_csv_path, index=False)
    
   # print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()


Enter the parent directory path: \\10.2.20.249\Lab Data\Joel Tencer\Agilent HPLC 2\SEC\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28\2AB-0401.D
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [ ]:
# v1.7
# changed where sequence name is taken from Sheet1 B15 to Sheet1 B5
# Add DAD signal for each?
# Look into full report save options, why was fileinfo1 sheet in one thing and not another, check all are the same
# Test with other column run data

import os
import xlrd
import pandas as pd

def extract_data_from_file(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')

    #Extract data from FileInfo1 sheet
    #fileinfo1_sheet = wb.sheet_by_name('FileInfo1')

    # Extract data from Signal sheet
    signal_sheet = wb.sheet_by_name('Signal')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):
        # Extract Method from Sheet1 cell B6
        method = sheet1.cell_value(rowx=5, colx=1)  # B6
        
        # Extract Sequence from Sheet1 cell B5
        sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
        sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right

        # Extract file name from FileInfo1 cell D2
        #file_name = fileinfo1_sheet.cell_value(rowx=1, colx=3)  # D2

        # Extract File Name from Signal Sheet H2
        file_name_text = signal_sheet.cell_value(rowx=1, colx=7)  # H2
        file_name = file_name_text.rsplit("\\", 2)[-1]
        
        
        # Initialize dictionary for current row data
        row_data = {}

        # Add a column named 'Sample' with value equal to the sample name
        row_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
        
        # Add a column named 'Peak' with value equal to the current loop number n
        row_data['Peak'] = row

        # Check if column 22 has a value, set area percent accordingly
        try:
            area_percent = int_results_sheet.cell_value(rowx=row, colx=22)
        except IndexError:
            area_percent = ""
        row_data['Area_Percent'] = area_percent
        
        # Add other data columns recursively
        for col, label in zip(range(4, 9), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)

        
        # Add Sequence, Method, and File Name to row data
        row_data['Sequence'] = sequence
        row_data['File Name'] = file_name
        row_data['Method'] = method

        # Append the row data to the list
        rows_data.append(row_data)

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)
   # print('First loop: ', int_results_df)

    return int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                int_results_df = extract_data_from_file(file_path)
                
                # Append the extracted data to the respective lists
                int_results_df_list.append(int_results_df)
                #print('Main Loop: ', int_results_df)

    # Concatenate all DataFrames into a single DataFrame
    int_results_df_combined = pd.concat(int_results_df_list, ignore_index=True)
    
    # Print the extracted DataFrame
   # print("\nIntResults Dataframe:")
   # print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "HPLC_Summary.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
   # print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()


In [1]:
# v1.6
# removed baseline, timestart, timeend, etc.
# Look into full report save options, why was fileinfo1 sheet in one thing and not another, check all are the same
# Test with other column run data

import os
import xlrd
import pandas as pd

def extract_data_from_file(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')

    #Extract data from FileInfo1 sheet
    #fileinfo1_sheet = wb.sheet_by_name('FileInfo1')

    # Extract data from Signal sheet
    signal_sheet = wb.sheet_by_name('Signal')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):
        # Extract Method from Sheet1 cell B6
        method = sheet1.cell_value(rowx=5, colx=1)  # B6
        
        # Extract Sequence from Sheet1 cell B5
        sequence_text = sheet1.cell_value(rowx=14, colx=1)  # B15
        sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right

        # Extract file name from FileInfo1 cell D2
        #file_name = fileinfo1_sheet.cell_value(rowx=1, colx=3)  # D2

        # Extract File Name from Signal Sheet H2
        file_name_text = signal_sheet.cell_value(rowx=1, colx=7)  # H2
        file_name = file_name_text.rsplit("\\", 2)[-1]
        
        
        # Initialize dictionary for current row data
        row_data = {}

        # Add a column named 'Sample' with value equal to the sample name
        row_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
        
        # Add a column named 'Peak' with value equal to the current loop number n
        row_data['Peak'] = row

        # Check if column 22 has a value, set area percent accordingly
        try:
            area_percent = int_results_sheet.cell_value(rowx=row, colx=22)
        except IndexError:
            area_percent = ""
        row_data['Area_Percent'] = area_percent
        
        # Add other data columns recursively
        for col, label in zip(range(4, 9), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)

        
        # Add Sequence, Method, and File Name to row data
        row_data['Sequence'] = sequence
        row_data['File Name'] = file_name
        row_data['Method'] = method

        # Append the row data to the list
        rows_data.append(row_data)

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)
   # print('First loop: ', int_results_df)

    return int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                int_results_df = extract_data_from_file(file_path)
                
                # Append the extracted data to the respective lists
                int_results_df_list.append(int_results_df)
                #print('Main Loop: ', int_results_df)

    # Concatenate all DataFrames into a single DataFrame
    int_results_df_combined = pd.concat(int_results_df_list, ignore_index=True)
    
    # Print the extracted DataFrame
   # print("\nIntResults Dataframe:")
   # print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "HPLC_Results_Summary.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
   # print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()


Enter the parent directory path: C:\Users\JoelTencer\Documents\HPLC\full report
WARNING *** file size (38961) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


IndexError: list index out of range

In [42]:
# v1.5
# mains has a loop that re-defines the same exact data unnecesarily, remove for clarity and speed
# Look into full report save options, why was fileinfo1 sheet in one thing and not another, check all are the same
# Test with other column run data

import os
import xlrd
import pandas as pd

def extract_data_from_file(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')

    #Extract data from FileInfo1 sheet
    #fileinfo1_sheet = wb.sheet_by_name('FileInfo1')

    # Extract data from Signal sheet
    signal_sheet = wb.sheet_by_name('Signal')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):
        # Extract Method from Sheet1 cell B6
        method = sheet1.cell_value(rowx=5, colx=1)  # B6
        
        # Extract Sequence from Sheet1 cell B5
        sequence_text = sheet1.cell_value(rowx=14, colx=1)  # B15
        sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right

        # Extract file name from FileInfo1 cell D2
        #file_name = fileinfo1_sheet.cell_value(rowx=1, colx=3)  # D2

        # Extract File Name from Signal Sheet H2
        file_name_text = signal_sheet.cell_value(rowx=1, colx=7)  # H2
        file_name = file_name_text.rsplit("\\", 2)[-1]
        
        
        # Initialize dictionary for current row data
        row_data = {}

        # Add a column named 'Sample' with value equal to the sample name
        row_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
        
        # Add a column named 'Peak' with value equal to the current loop number n
        row_data['Peak'] = row

        # Check if column 22 has a value, set area percent accordingly
        try:
            area_percent = int_results_sheet.cell_value(rowx=row, colx=22)
        except IndexError:
            area_percent = ""
        row_data['Area_Percent'] = area_percent
        
        # Add other data columns recursively
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)

        
        # Add Sequence, Method, and File Name to row data
        row_data['Sequence'] = sequence
        row_data['File Name'] = file_name
        row_data['Method'] = method

        # Append the row data to the list
        rows_data.append(row_data)

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)
   # print('First loop: ', int_results_df)

    return int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                int_results_df = extract_data_from_file(file_path)
                
                # Append the extracted data to the respective lists
                int_results_df_list.append(int_results_df)
                #print('Main Loop: ', int_results_df)

    # Concatenate all DataFrames into a single DataFrame
    int_results_df_combined = pd.concat(int_results_df_list, ignore_index=True)
    
    # Print the extracted DataFrame
   # print("\nIntResults Dataframe:")
   # print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "HPLC_Results_Summary.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
   # print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()


Enter the parent directory path:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 05 03 769-773 RERUN 2024-05-03 12-44-02


WARNING *** file size (36491) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (33958) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (34801) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (33958) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (35654) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (35654) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (35654) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero

In [65]:
import os
import xlrd
import pandas as pd

def extract_data_from_file(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):
        # Initialize dictionary for current row data
        row_data = {}

        # Add a column named 'Sample' with value equal to the sample name
        row_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
        
        # Add a column named 'Peak' with value equal to the current loop number n
        row_data['Peak'] = row

        # Extract Method from Sheet1 cell B6
        method = sheet1.cell_value(rowx=5, colx=1)  # B6
        
        # Extract Sequence from Sheet1 cell B5
       # sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
       # sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right
            

        # Check if column 22 has a value, set area percent accordingly
        try:
            area_percent = int_results_sheet.cell_value(rowx=row, colx=22)
        except IndexError:
            area_percent = ""
        row_data['Area_Percent'] = area_percent
        
        # Add other data columns recursively
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)

        # Append the row data to the list
        rows_data.append(row_data)

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    return int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    # for root, dirs, files in os.walk(parent_directory):
    #    for file in files:
    #        if file.endswith(".xls"):
    #            file_path = os.path.join(root, file)
    #            
    #            # Extract data from specific cells
    #            int_results_df = extract_data_from_file(file_path)
    #            
                # Append the extracted data to the list
                int_results_df_list.append(int_results_df)

    # Concatenate all DataFrames into a single DataFrame
    int_results_df_combined = pd.concat(int_results_df_list, ignore_index=True)
    
    # Print the extracted DataFrame
    print("\nIntResults Dataframe:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "int_results.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()


Enter the parent directory path:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28


WARNING *** file size (32454) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


IndexError: list index out of range

In [63]:
# not iterating on right cell for file name, sequence, and method but has recursive loops setup

import os
import xlrd
import pandas as pd

def extract_data_from_file(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):
        # Initialize dictionary for current row data
        row_data = {}

        # Add a column named 'Sample' with value equal to the sample name
        row_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
        
        # Add a column named 'Peak' with value equal to the current loop number n
        row_data['Peak'] = row

        # Extract Sequence, Method, and File Name
        for col, label in zip([1, 1, 1], ['Sequence', 'Method', 'File Name']):
            cell_value = sheet1.cell_value(rowx=row, colx=col)
            if col == 1:  # For Sequence and Method
                cell_value = cell_value.rsplit("\\", 2)[-2] if "\\" in cell_value else cell_value
            row_data[label] = cell_value

        # Check if column 22 has a value, set area percent accordingly
        try:
            area_percent = int_results_sheet.cell_value(rowx=row, colx=22)
        except IndexError:
            area_percent = ""
        row_data['Area_Percent'] = area_percent
        
        # Add other data columns recursively
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)

        # Append the row data to the list
        rows_data.append(row_data)

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    return int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                int_results_df = extract_data_from_file(file_path)
                
                # Append the extracted data to the list
                int_results_df_list.append(int_results_df)

    # Concatenate all DataFrames into a single DataFrame
    int_results_df_combined = pd.concat(int_results_df_list, ignore_index=True)
    
    # Print the extracted DataFrame
    print("\nIntResults Dataframe:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "int_results.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()


Enter the parent directory path:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28


WARNING *** file size (32454) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero

TypeError: argument of type 'float' is not iterable

In [72]:
# need to change so it iterates through file name, method, and sequence somehow in nested or double loop

import os
import xlrd
import pandas as pd

def extract_data_from_file(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):
        # Extract Method from Sheet1 cell B6
        method = sheet1.cell_value(rowx=5, colx=1)  # B6
        
        # Extract Sequence from Sheet1 cell B5
        sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
        sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right

        # Extract file name from Sheet1 cell B33
        file_name_text = sheet1.cell_value(rowx=32, colx=1)  # B33
        file_name = file_name_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right
        
        # Initialize dictionary for current row data
        row_data = {}

        # Add a column named 'Sample' with value equal to the sample name
        row_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
        
        # Add a column named 'Peak' with value equal to the current loop number row
        row_data['Peak'] = row
        
        # Add Sequence, Method, and File Name to row data
        # row_data['Sequence'] = sequence[row]
        # row_data['Method' = method
        # row_data['File Name'] = file_name


        # Check if column 22 has a value, set area percent accordingly
        try:
            area_percent = int_results_sheet.cell_value(rowx=row, colx=22)
        except IndexError:
            area_percent = ""
        row_data['Area_Percent'] = area_percent
        
        # Add other data columns recursively
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)
            # Add Sequence, Method, and File Name to row data
            row_data['Sequence'] = sequence[col]
            row_data['Method'] = method[col]
            row_data['File Name'] = file_name[col]

        # Append the row data to the list
        rows_data.append(row_data)

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    return int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                int_results_df = extract_data_from_file(file_path)
                
                # Append the extracted data to the list
                int_results_df_list.append(int_results_df)

    # Concatenate all DataFrames into a single DataFrame
    int_results_df_combined = pd.concat(int_results_df_list, ignore_index=True)
    
    # Print the extracted DataFrame
    print("\nIntResults Dataframe:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "int_results.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()


Enter the parent directory path:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28


WARNING *** file size (32454) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


IndexError: string index out of range

In [74]:
import os
import xlrd
import pandas as pd

def extract_data_from_file(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):
        # Extract Method from Sheet1 cell B6
        method = sheet1.cell_value(rowx=5, colx=1)  # B6
        
        # Extract Sequence from Sheet1 cell B5
        sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
        sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right

        # Extract file name from Sheet1 cell B33
        file_name_text = sheet1.cell_value(rowx=32, colx=1)  # B33
        file_name = file_name_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right
        
        # Initialize dictionary for current row data
        row_data = {}

        # Add a column named 'Sample' with value equal to the sample name
        row_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
        
        # Add a column named 'Peak' with value equal to the current loop number n
        row_data['Peak'] = row

        # Add Sequence, Method, and File Name to row data
        row_data['Sequence'] = sequence
        row_data['Method'] = method
        row_data['File Name'] = file_name

        # Check if column 22 has a value, set area percent accordingly
        try:
            area_percent = int_results_sheet.cell_value(rowx=row, colx=22)
        except IndexError:
            area_percent = ""
        row_data['Area_Percent'] = area_percent
        
        # Add other data columns recursively
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)

        # Append the row data to the list
        rows_data.append(row_data)

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    return int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                int_results_df = extract_data_from_file(file_path)
                
                # Append the extracted data to the respective lists
                int_results_df_list.append(int_results_df)

    # Concatenate all DataFrames into a single DataFrame
    int_results_df_combined = pd.concat(int_results_df_list, ignore_index=True)
    
    # Print the extracted DataFrame
    print("\nIntResults Dataframe:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "int_results.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()


Enter the parent directory path:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28


WARNING *** file size (32454) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero

In [57]:
#not extracting file name method and sequence right only taking from first

import os
import xlrd
import pandas as pd

def extract_data_from_cells(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Initialize dictionaries to store extracted data
    sample_data = {}
    int_results_data = {}

    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    sample_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26

    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):
        # Extract Method from Sheet1 cell B6
        method = sheet1.cell_value(rowx=5, colx=1)  # B6
        
        # Extract Sequence from Sheet1 cell B5
        sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
        sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right

        # Extract file name from Sheet1 cell B33
        file_name_text = sheet1.cell_value(rowx=32, colx=1)  # B33
        file_name = file_name_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right
        
        # Initialize dictionary for current row data
        row_data = {}

        # Add a column named 'Sample' with value equal to the sample name
        int_results_data['Sample'] = sample_data['Sample']
        
        # Add a column named 'Peak' with value equal to the current loop number n
        int_results_data['Peak'] = row

        # Add Sequence, Method, and File Name to row data
        int_results_data['Sequence'] = sequence
        int_results_data['Method'] = method
        int_results_data['File Name'] = file_name

        # Check if column 22 has a value, set area percent accordingly
        try:
            area_percent = int_results_sheet.cell_value(rowx=row, colx=22)
        except IndexError:
            area_percent = ""
        int_results_data['Area_Percent'] = area_percent
        
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)

        
        # Append the row data to the list
        rows_data.append({**int_results_data, **row_data})

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    return sample_data, int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                sample_data, int_results_df = extract_data_from_cells(file_path)
                
                # Append the extracted data to the respective lists
                sample_data_list.append(sample_data)
                int_results_df_list.append(int_results_df)

    # Convert the extracted data into dataframes
    sample_df = pd.DataFrame(sample_data_list)
    int_results_df_combined = pd.concat(int_results_df_list)
    
    # Print the extracted dataframes
    print("Sample Dataframe:")
    print(sample_df)
    print("\nIntResults Dataframe:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "int_results.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()

Enter the parent directory path:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28


WARNING *** file size (32454) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\JoelTencer\\Documents\\Agilent HPLC 2\\SEC\\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28\\int_results.csv'

In [52]:
# Add file name and DAD signal

import os
import xlrd
import pandas as pd

def extract_data_from_cells(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Initialize dictionaries to store extracted data
    sample_data = {}
    int_results_data = {}

    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
   # fileinfo1 = wb.sheet_by_name('FileInfo1')
    sample_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26

    # Extract Method from Sheet1 cell B6
    method = sheet1.cell_value(rowx=5, colx=1)  # B6
    
    # Extract Sequence from Sheet1 cell B5
    sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
    sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right

    # Extract file name from Sheet1 cell B33
    file_name_text = sheet1.cell_value(rowx=32, colx=1)  # B33
    file_name = file_name_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):

        # Add a column named 'Sample' with value equal to the sample name
        int_results_data['Sample'] = sample_data['Sample']
        
        # Add a column named 'Peak' with value equal to the current loop number n
        int_results_data['Peak'] = row

        # Add a column named 'Area Perccent' with value equal to the peak's area percent
        # Add the 'Area_Percent' column after 'Peak' column
        
        # Check if column 22 has a value, set area percent accordingly
        try:
            area_percent = int_results_sheet.cell_value(rowx=row, colx=22)
        except IndexError:
            area_percent = ""
        int_results_data['Area_Percent'] = area_percent
        
        sheet_data = {}
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            sheet_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)

        #Make a third data frame for data to add after main data columns
        meta_data = {}
        meta_data['Sequence'] = sequence
        meta_data['Method'] = method
        meta_data['File Name'] = file_name
        
        # Append the row data to the list
        rows_data.append({**int_results_data, **sheet_data, **meta_data})

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    return sample_data, int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                sample_data, int_results_df = extract_data_from_cells(file_path)
                
                # Append the extracted data to the respective lists
                sample_data_list.append(sample_data)
                int_results_df_list.append(int_results_df)

    # Convert the extracted data into dataframes
    sample_df = pd.DataFrame(sample_data_list)
    int_results_df_combined = pd.concat(int_results_df_list)
    
    # Print the extracted dataframes
    print("Sample Dataframe:")
    print(sample_df)
    print("\nIntResults Dataframe:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "int_results.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()


Enter the parent directory path:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28


WARNING *** file size (32454) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero

In [49]:
# WORKING SCRIPTS!

import os
import xlrd
import pandas as pd

def extract_data_from_cells(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Initialize dictionaries to store extracted data
    sample_data = {}
    int_results_data = {}

    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    sample_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26

    # Extract Method from Sheet1 cell B6
    method = sheet1.cell_value(rowx=5, colx=1)  # B6
    
    # Extract Sequence from Sheet1 cell B5
    sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
    sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right

    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):

        # Add a column named 'Sample' with value equal to the sample name
        int_results_data['Sample'] = sample_data['Sample']
        
        # Add a column named 'Peak' with value equal to the current loop number n
        int_results_data['Peak'] = row

        # Add a column named 'Area Perccent' with value equal to the peak's area percent
        # Add the 'Area_Percent' column after 'Peak' column
        
        # Check if column 22 has a value, set area percent accordingly
        try:
            area_percent = int_results_sheet.cell_value(rowx=row, colx=22)
        except IndexError:
            area_percent = ""
        int_results_data['Area_Percent'] = area_percent
        
        sheet_data = {}
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            sheet_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)

        #Make a third data frame for data to add after main data columns
        meta_data = {}
        meta_data['Sequence'] = sequence
        meta_data['Method'] = method
        
        # Append the row data to the list
        rows_data.append({**int_results_data, **sheet_data, **meta_data})

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    return sample_data, int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                sample_data, int_results_df = extract_data_from_cells(file_path)
                
                # Append the extracted data to the respective lists
                sample_data_list.append(sample_data)
                int_results_df_list.append(int_results_df)

    # Convert the extracted data into dataframes
    sample_df = pd.DataFrame(sample_data_list)
    int_results_df_combined = pd.concat(int_results_df_list)
    
    # Print the extracted dataframes
    print("Sample Dataframe:")
    print(sample_df)
    print("\nIntResults Dataframe:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "int_results.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()


Enter the parent directory path:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28


WARNING *** file size (32454) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero

In [30]:
def extract_data_from_cells(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Initialize dictionaries to store extracted data
    sample_data = {}
    int_results_data = {}

    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    sample_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
    
    # Extract Method from Sheet1 cell B6
    method = sheet1.cell_value(rowx=5, colx=1)  # B6
    
    # Extract Sequence from Sheet1 cell B5
    sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
    sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Determine the number of rows and columns in the IntResults sheet
    num_rows = int_results_sheet.nrows
    num_cols = int_results_sheet.ncols
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to the total number of rows
    for row in range(1, num_rows):
        # Check if the row index is within the range of available rows
        if row < num_rows:
            # Add a column named 'Peak' with value equal to the current loop number n
            int_results_data['Sample'] = sample_data['Sample']
            int_results_data['Peak'] = row
            
            # Add the 'Area_Percent' column after 'Peak' column
            if 0 <= 22 < num_cols and int_results_sheet.cell_type(rowx=row, colx=22) != xlrd.XL_CELL_EMPTY:
                area_percent = int_results_sheet.cell_value(rowx=row, colx=22)  # Assuming area percent is in column 22
            else:
                area_percent = ""  # Set area percent to an empty string if it has no value
            int_results_data['Area_Percent'] = area_percent
            
            row_data = {}
            for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
                row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)
            
            # Append the row data to the list
            rows_data.append({**int_results_data, **row_data})

    # Append the 'Sequence' column to the end of the rows_data list
    for row_data in rows_data:
        row_data['Sequence'] = sequence
        
        # Append the 'Method' column to the end of the rows_data list
        row_data['Method'] = method

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    return sample_data, int_results_df
def main():
    # Prompt user to input the parent directory
    parent_directory = input("Paste the parent folder directory path from Windows Explorer: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                sample_data, int_results_df = extract_data_from_cells(file_path)
                
                # Append the extracted data to the respective lists
                sample_data_list.append(sample_data)
                int_results_df_list.append(int_results_df)

    # Convert the extracted data into dataframes
    sample_df = pd.DataFrame(sample_data_list)
    int_results_df_combined = pd.concat(int_results_df_list)
    
    # Print the extracted dataframes
    print("Sample Dataframe:")
    print(sample_df)
    print("\nIntResults Dataframe:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "HPLC_Data_Extract.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()



Paste the parent folder directory path from Windows Explorer:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28


WARNING *** file size (32454) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero

In [28]:
def extract_data_from_cells(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Initialize dictionaries to store extracted data
    sample_data = {}
    int_results_data = {}

    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    sample_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
    
    # Extract Method from Sheet1 cell B6
    method = sheet1.cell_value(rowx=5, colx=1)  # B6
    
    # Extract Sequence from Sheet1 cell B5
    sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
    sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the number of rows in the IntResults sheet
    num_rows = int_results_sheet.nrows
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to the total number of rows
    for row in range(1, num_rows):
        # Check if the row index is within the range of available rows
        if row < num_rows:
            # Add a column named 'Peak' with value equal to the current loop number n
            int_results_data['Sample'] = sample_data['Sample']
            int_results_data['Peak'] = row
            
            # Add the 'Area_Percent' column after 'Peak' column
            if int_results_sheet.cell_type(rowx=row, colx=22) == xlrd.XL_CELL_EMPTY:
                area_percent = ""
            else:
                area_percent = int_results_sheet.cell_value(rowx=row, colx=22)  # Assuming area percent is in column 22
            int_results_data['Area_Percent'] = area_percent
            
            row_data = {}
            for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
                row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)
            
            # Append the row data to the list
            rows_data.append({**int_results_data, **row_data})

    # Append the 'Sequence' column to the end of the rows_data list
    for row_data in rows_data:
        row_data['Sequence'] = sequence
        
        # Append the 'Method' column to the end of the rows_data list
        row_data['Method'] = method

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    return sample_data, int_results_df
    
def main():
    # Prompt user to input the parent directory
    parent_directory = input("Paste the parent folder directory path from Windows Explorer: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                sample_data, int_results_df = extract_data_from_cells(file_path)
                
                # Append the extracted data to the respective lists
                sample_data_list.append(sample_data)
                int_results_df_list.append(int_results_df)

    # Convert the extracted data into dataframes
    sample_df = pd.DataFrame(sample_data_list)
    int_results_df_combined = pd.concat(int_results_df_list)
    
    # Print the extracted dataframes
    print("Sample Dataframe:")
    print(sample_df)
    print("\nIntResults Dataframe:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "HPLC_Data_Extract.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()



Paste the parent folder directory path from Windows Explorer:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28\2AG-0901.D


WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


IndexError: array index out of range

In [25]:
#

def extract_data_from_cells(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Initialize dictionaries to store extracted data
    sample_data = {}
    int_results_data = {}

    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    sample_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
    
    # Extract Method from Sheet1 cell B6
    method = sheet1.cell_value(rowx=5, colx=1)  # B6
    
    # Extract Sequence from Sheet1 cell B5
    sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
    sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):

        # Add a column named 'Peak' with value equal to the current loop number n
        int_results_data['Sample'] = sample_data['Sample']
        int_results_data['Peak'] = row
        
        # Add the 'Area_Percent' column after 'Peak' column
        area_percent = int_results_sheet.cell_value(rowx=row, colx=22)  # Assuming area percent is in column 22
        if area_percent == "":
            area_percent = ""  # Set area percent to an empty string if it has no value
        int_results_data['Area_Percent'] = area_percent
        
        row_data = {}
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)
        
        # Append the row data to the list
        rows_data.append({**int_results_data, **row_data})

    # Append the 'Sequence' column to the end of the rows_data list
    for row_data in rows_data:
        row_data['Sequence'] = sequence
        
        # Append the 'Method' column to the end of the rows_data list
        row_data['Method'] = method

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    return sample_data, int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Paste the parent folder directory path from Windows Explorer: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                sample_data, int_results_df = extract_data_from_cells(file_path)
                
                # Append the extracted data to the respective lists
                sample_data_list.append(sample_data)
                int_results_df_list.append(int_results_df)

    # Convert the extracted data into dataframes
    sample_df = pd.DataFrame(sample_data_list)
    int_results_df_combined = pd.concat(int_results_df_list)
    
    # Print the extracted dataframes
    print("Sample Dataframe:")
    print(sample_df)
    print("\nIntResults Dataframe:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "HPLC_Data_Extract.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()



Paste the parent folder directory path from Windows Explorer:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28


WARNING *** file size (32454) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


IndexError: list index out of range

In [24]:
# This is v2, only works with full report #NOT FULL, SHORT OR DETAIL, FILEINFO SHEET WEIRD

def extract_data_from_cells(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Initialize dictionaries to store extracted data
    sample_data = {}
    int_results_data = {}

    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    sample_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
    
    # Extract Method from Sheet1 cell B6
    method = sheet1.cell_value(rowx=5, colx=1)  # B6
    
    # Extract Sequence from Sheet1 cell B5
    sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
    sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):

        # Add a column named 'Peak' with value equal to the current loop number n
        int_results_data['Sample'] = sample_data['Sample']
        int_results_data['Peak'] = row
        
        # Add the 'Area_Percent' column after 'Peak' column
        area_percent = int_results_sheet.cell_value(rowx=row, colx=22)  # Assuming area percent is in column 22
        int_results_data['Area_Percent'] = area_percent
        
        row_data = {}
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)
        
        # Append the row data to the list
        rows_data.append({**int_results_data, **row_data})

    # Append the 'Sequence' column to the end of the rows_data list
    for row_data in rows_data:
        row_data['Sequence'] = sequence
        
        # Append the 'Method' column to the end of the rows_data list
        row_data['Method'] = method

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    return sample_data, int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Paste the parent folder directory path from Windows Explorer: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                sample_data, int_results_df = extract_data_from_cells(file_path)
                
                # Append the extracted data to the respective lists
                sample_data_list.append(sample_data)
                int_results_df_list.append(int_results_df)

    # Convert the extracted data into dataframes
    sample_df = pd.DataFrame(sample_data_list)
    int_results_df_combined = pd.concat(int_results_df_list)
    
    # Print the extracted dataframes
    print("Sample Dataframe:")
    print(sample_df)
    print("\nIntResults Dataframe:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "HPLC_Data_Extract.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()


Paste the parent folder directory path from Windows Explorer:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28


WARNING *** file size (32454) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


IndexError: list index out of range

In [20]:
def extract_data_from_cells(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Initialize dictionaries to store extracted data
    sample_data = {}
    int_results_data = {}

    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    sample_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
    
    # Extract Method from Sheet1 cell B6
    method = sheet1.cell_value(rowx=5, colx=1)  # B6
    
    # Extract Sequence from Sheet1 cell B5
    sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
    sequence = sequence_text.rsplit("\\", 2)[-2]  # Extract text between two backslashes starting from the right
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):

        # Add a column named 'Peak' with value equal to the current loop number n
        int_results_data['Sample'] = sample_data['Sample']
        int_results_data['Peak'] = row
        
        # Add the 'Area_Percent' column after 'Peak' column
        area_percent = int_results_sheet.cell_value(rowx=row, colx=22)  # Assuming area percent is in column 22
        int_results_data['Area_Percent'] = area_percent
        
        row_data = {}
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)
        
        # Append the row data to the list
        rows_data.append({**int_results_data, **row_data})

        # Append the 'Sequence' column to the end of the rows_data list
        int_results_data['Sequence'] = sequence
        
        # Append the 'Method' column to the end of the rows_data list
        int_results_data['Method'] = method

        # Create DataFrame from the list of row data
        int_results_df = pd.DataFrame(rows_data)

    return sample_data, int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Paste the parent folder directory path from Windows Explorer: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                sample_data, int_results_df = extract_data_from_cells(file_path)
                
                # Append the extracted data to the respective lists
                sample_data_list.append(sample_data)
                int_results_df_list.append(int_results_df)

    # Convert the extracted data into dataframes
    sample_df = pd.DataFrame(sample_data_list)
    int_results_df_combined = pd.concat(int_results_df_list)
    
    # Print the extracted dataframes
    print("Sample Dataframe:")
    print(sample_df)
    print("\nIntResults Dataframe:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "HPLC_Data_Extract.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()

Paste the parent folder directory path from Windows Explorer:  C:\Users\JoelTencer\Documents\HPLC\full report


WARNING *** file size (38961) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Sample Dataframe:
    Sample
0  770 1A3

IntResults Dataframe:
    Sample  Peak  Area_Percent   RetTime         Area     Height     Width  \
0  770 1A3     1      0.427019  2.533667     4.396246   0.280334  0.234959   
1  770 1A3     2      0.604369  3.266259     6.222103   0.374421  0.231876   
2  770 1A3     3     97.946593  3.727843  1008.380127  61.024811  0.238781   
3  770 1A3     4      0.225631  5.240099     2.322921   0.355774  0.092174   
4  770 1A3     5      0.372004  5.423666     3.829855   0.377986  0.148210   
5  770 1A3     6      0.186258  5.902758     1.917563   0.146571  0.164633   
6  770 1A3     7      0.238126  6.039222     2.451552   0.155675  0.219726   

   Symmetry  Baseline  TimeStart  LevelStart  BaselineStart   TimeEnd  \
0  0.571415 -0.010205   2.277000    0.000000      -0.021458  2.873667   
1  1.322650  0.021912  

In [7]:
def extract_data_from_cells(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Initialize dictionaries to store extracted data
    sample_data = {}
    int_results_data = {}

    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    sample_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
    
    # Extract Sequence from Sheet1 cell B5
    sequence_text = sheet1.cell_value(rowx=4, colx=1)  # B5
    sequence = sequence_text.split("\\")[1]  # Extract text between two backslashes
    int_results_data['Sequence'] = sequence
    
    # Extract Method from Sheet1 cell B6
    method = sheet1.cell_value(rowx=5, colx=1)  # B6
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):

        # Add a column named 'Peak' with value equal to the current loop number n
        int_results_data['Sample'] = sample_data['Sample']
        int_results_data['Peak'] = row
        
        # Add the 'Area_Percent' column after 'Peak' column
        area_percent = int_results_sheet.cell_value(rowx=row, colx=22)  # Assuming area percent is in column 22
        int_results_data['Area_Percent'] = area_percent
        
        row_data = {}
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)
        
        # Append the row data to the list
        rows_data.append({**int_results_data, **row_data})

    # Append the 'Method' column to the end of the rows_data list
    for row in rows_data:
        row['Method'] = method

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    return sample_data, int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Paste the parent folder directory path from Windows Explorer: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                sample_data, int_results_df = extract_data_from_cells(file_path)
                
                # Append the extracted data to the respective lists
                sample_data_list.append(sample_data)
                int_results_df_list.append(int_results_df)

    # Convert the extracted data into dataframes
    sample_df = pd.DataFrame(sample_data_list)
    int_results_df_combined = pd.concat(int_results_df_list)
    
    # Print the extracted dataframes
    print("Sample Dataframe:")
    print(sample_df)
    print("\nIntResults Dataframe:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "int_results.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()


Paste the parent folder directory path from Windows Explorer:  C:\Users\JoelTencer\Documents\HPLC\full report


WARNING *** file size (38961) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Sample Dataframe:
    Sample
0  770 1A3

IntResults Dataframe:
  Sequence   Sample  Peak  Area_Percent   RetTime         Area     Height  \
0   Chem32  770 1A3     1      0.427019  2.533667     4.396246   0.280334   
1   Chem32  770 1A3     2      0.604369  3.266259     6.222103   0.374421   
2   Chem32  770 1A3     3     97.946593  3.727843  1008.380127  61.024811   
3   Chem32  770 1A3     4      0.225631  5.240099     2.322921   0.355774   
4   Chem32  770 1A3     5      0.372004  5.423666     3.829855   0.377986   
5   Chem32  770 1A3     6      0.186258  5.902758     1.917563   0.146571   
6   Chem32  770 1A3     7      0.238126  6.039222     2.451552   0.155675   

      Width  Symmetry  Baseline  TimeStart  LevelStart  BaselineStart  \
0  0.234959  0.571415 -0.010205   2.277000    0.000000      -0.021458   
1  0.231876  1.322650  0.021912

In [6]:
def extract_data_from_cells(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Initialize dictionaries to store extracted data
    sample_data = {}
    int_results_data = {}

    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    sample_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
    
    # Extract Method from Sheet1 cell B6
    method = sheet1.cell_value(rowx=5, colx=1)  # B6
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):

        # Add a column named 'Peak' with value equal to the current loop number n
        int_results_data['Sample'] = sample_data['Sample']
        int_results_data['Peak'] = row
        
        # Add the 'Area_Percent' column after 'Peak' column
        area_percent = int_results_sheet.cell_value(rowx=row, colx=22)  # Assuming area percent is in column 22
        int_results_data['Area_Percent'] = area_percent
        
        row_data = {}
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)
        
        # Append the row data to the list
        rows_data.append({**int_results_data, **row_data})
    
    # Append the 'Method' column to the end of the rows_data list
    for row in rows_data:
        row['Method'] = method

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    return sample_data, int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Paste the parent folder directory path from Windows Explorer: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                sample_data, int_results_df = extract_data_from_cells(file_path)
                
                # Append the extracted data to the respective lists
                sample_data_list.append(sample_data)
                int_results_df_list.append(int_results_df)

    # Convert the extracted data into dataframes
    sample_df = pd.DataFrame(sample_data_list)
    int_results_df_combined = pd.concat(int_results_df_list)
    
    # Print the extracted dataframes
    print("Sample Dataframe:")
    print(sample_df)
    print("\nIntResults Dataframe:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "int_results.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()


Paste the parent folder directory path from Windows Explorer:  C:\Users\JoelTencer\Documents\HPLC\full report


WARNING *** file size (38961) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Sample Dataframe:
    Sample
0  770 1A3

IntResults Dataframe:
    Sample  Peak  Area_Percent   RetTime         Area     Height     Width  \
0  770 1A3     1      0.427019  2.533667     4.396246   0.280334  0.234959   
1  770 1A3     2      0.604369  3.266259     6.222103   0.374421  0.231876   
2  770 1A3     3     97.946593  3.727843  1008.380127  61.024811  0.238781   
3  770 1A3     4      0.225631  5.240099     2.322921   0.355774  0.092174   
4  770 1A3     5      0.372004  5.423666     3.829855   0.377986  0.148210   
5  770 1A3     6      0.186258  5.902758     1.917563   0.146571  0.164633   
6  770 1A3     7      0.238126  6.039222     2.451552   0.155675  0.219726   

   Symmetry  Baseline  TimeStart  LevelStart  BaselineStart   TimeEnd  \
0  0.571415 -0.010205   2.277000    0.000000      -0.021458  2.873667   
1  1.322650  0.021912  

In [2]:
## add area percent from full report xls file

import os
import xlrd
import pandas as pd

def extract_data_from_cells(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Initialize dictionaries to store extracted data
    sample_data = {}
    int_results_data = {}

    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    sample_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):

        
        
        # Add a column named 'Peak' with value equal to the current loop number n
        int_results_data['Sample'] = sample_data['Sample']
        int_results_data['Peak'] = row
        
        # Add the 'Area_Percent' column after 'Peak' column
        area_percent = int_results_sheet.cell_value(rowx=row, colx=22)  # Assuming area percent is in column 22
        int_results_data['Area_Percent'] = area_percent
        
        row_data = {}
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)
        
        # Append the row data to the list
        rows_data.append({**int_results_data, **row_data})

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    return sample_data, int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Paste the parent folder directory path from Windows Explorer: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                sample_data, int_results_df = extract_data_from_cells(file_path)
                
                # Append the extracted data to the respective lists
                sample_data_list.append(sample_data)
                int_results_df_list.append(int_results_df)

    # Convert the extracted data into dataframes
    sample_df = pd.DataFrame(sample_data_list)
    int_results_df_combined = pd.concat(int_results_df_list)
    
    # Print the extracted dataframes
    print("Sample Dataframe:")
    print(sample_df)
    print("\nIntResults Dataframe:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "int_results.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()


Paste the parent folder directory path from Windows Explorer:  C:\Users\JoelTencer\Documents\HPLC\full report


WARNING *** file size (38961) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Sample Dataframe:
    Sample
0  770 1A3

IntResults Dataframe:
    Sample  Peak  Area_Percent   RetTime         Area     Height     Width  \
0  770 1A3     1      0.427019  2.533667     4.396246   0.280334  0.234959   
1  770 1A3     2      0.604369  3.266259     6.222103   0.374421  0.231876   
2  770 1A3     3     97.946593  3.727843  1008.380127  61.024811  0.238781   
3  770 1A3     4      0.225631  5.240099     2.322921   0.355774  0.092174   
4  770 1A3     5      0.372004  5.423666     3.829855   0.377986  0.148210   
5  770 1A3     6      0.186258  5.902758     1.917563   0.146571  0.164633   
6  770 1A3     7      0.238126  6.039222     2.451552   0.155675  0.219726   

   Symmetry  Baseline  TimeStart  LevelStart  BaselineStart   TimeEnd  
0  0.571415 -0.010205   2.277000    0.000000      -0.021458  2.873667  
1  1.322650  0.021912   2

In [31]:
# WORKING SCRIPTS!

import os
import xlrd
import pandas as pd

def extract_data_from_cells(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Initialize dictionaries to store extracted data
    sample_data = {}
    int_results_data = {}

    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    sample_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 1 up to row n
    for row in range(1, n + 1):
        # Add a column named 'Peak' with value equal to the current loop number n
        int_results_data['Sample'] = sample_data['Sample']
        int_results_data['Peak'] = row
        
        row_data = {}
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)
        # Append the row data to the list
        rows_data.append({**int_results_data, **row_data})

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    return sample_data, int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Recursively search for .xls files in the parent directory and its subdirectories
    for root, dirs, files in os.walk(parent_directory):
        for file in files:
            if file.endswith(".xls"):
                file_path = os.path.join(root, file)
                
                # Extract data from specific cells
                sample_data, int_results_df = extract_data_from_cells(file_path)
                
                # Append the extracted data to the respective lists
                sample_data_list.append(sample_data)
                int_results_df_list.append(int_results_df)

    # Convert the extracted data into dataframes
    sample_df = pd.DataFrame(sample_data_list)
    int_results_df_combined = pd.concat(int_results_df_list)
    
    # Print the extracted dataframes
    print("Sample Dataframe:")
    print(sample_df)
    print("\nIntResults Dataframe:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "int_results.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()


Enter the parent directory path:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28


WARNING *** file size (32454) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (32401) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (31188) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero

In [47]:
import os
import xlrd
import pandas as pd

def extract_data_from_cells(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Initialize dictionaries to store extracted data
    sample_data = {}
    int_results_data = {}

    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    sample_data['Sample'] = sheet1.cell_value(rowx=25, colx=1)  # B26
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 2 up to row 2 + n
    for row in range(1, 1 + n):
        # Add a column named 'Peak' with value equal to the current loop number n
        int_results_data['Sample'] = sample_data['Sample']
        int_results_data['Peak'] = row
        
        row_data = {}
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)
        # Append the row data to the list
        rows_data.append({**int_results_data, **row_data})

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    return sample_data, int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Loop through each file in the directory
    for filename in os.listdir(parent_directory):
        # Check if the file ends with ".xls"
        if filename.endswith(".xls"):
            file_path = os.path.join(parent_directory, filename)
            
            # Extract data from specific cells
            sample_data, int_results_df = extract_data_from_cells(file_path)
            
            # Append the extracted data to the respective lists
            sample_data_list.append(sample_data)
            int_results_df_list.append(int_results_df)

    # Convert the extracted data into dataframes
    sample_df = pd.DataFrame(sample_data_list)
    int_results_df_combined = pd.concat(int_results_df_list)
    
    # Print the extracted dataframes
    print("Sample Dataframe:")
    print(sample_df)
    print("\nIntResults Dataframe:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "int_results.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()


Enter the parent directory path:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28


ValueError: No objects to concatenate

In [40]:
import os
import xlrd
import pandas as pd

def extract_data_from_cells(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Initialize dictionaries to store extracted data
    sample_data = {}
    int_results_data = {}

    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    sample_data['SampleName'] = sheet1.cell_value(rowx=25, colx=1)  # B26
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 2 up to row 2 + n
    for row in range(1, 1 + n):
        # Add a column named 'Peak' with value equal to the current loop number n
        int_results_data['Peak'] = row - 1
        
        row_data = {}
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)
        # Append the row data to the list
        rows_data.append({**int_results_data, **row_data})

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    return sample_data, int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Loop through each file in the directory
    for filename in os.listdir(parent_directory):
        # Check if the file ends with ".xls"
        if filename.endswith(".xls"):
            file_path = os.path.join(parent_directory, filename)
            
            # Extract data from specific cells
            sample_data, int_results_df = extract_data_from_cells(file_path)
            
            # Append the extracted data to the respective lists
            sample_data_list.append(sample_data)
            int_results_df_list.append(int_results_df)

    # Convert the extracted data into dataframes
    sample_df = pd.DataFrame(sample_data_list)
    int_results_df_combined = pd.concat(int_results_df_list)
    
    # Print the extracted dataframes
    print("Sample Dataframe:")
    print(sample_df)
    print("\nIntResults Dataframe:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "int_results.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()


Enter the parent directory path:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28\2AJ-1401.D


WARNING *** file size (36244) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Sample Dataframe:
  SampleName
0     ladder

IntResults Dataframe:
   Peak   RetTime         Area     Height     Width  Symmetry   Baseline  \
0     0  2.629555   106.284401   6.798275  0.228397  1.765334  -2.602099   
1     1  2.962707   455.443756  25.940151  0.254837  0.867126  -4.687346   
2     2  3.230931   264.975800  16.417297  0.226300  0.294813  -6.366199   
3     3  3.722415  1017.263245  36.858269  0.373992  0.476035  -9.442472   
4     4  4.433238   647.048218  34.472748  0.258982  0.930943 -13.891619   
5     5  5.141259   215.237579   5.513213  0.467470  5.577894  -5.279715   
6     6  5.902357   122.673340   7.320348  0.257508  1.150406   0.083779   

   TimeStart  LevelStart  BaselineStart   TimeEnd  
0   2.212000    0.000000       0.011444  2.730381  
1   2.730381    6.832913      -3.233181  3.166339  
2   3.166339   15.565557 

In [35]:
import os
import xlrd
import pandas as pd

def extract_data_from_cells(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Initialize dictionaries to store extracted data
    sample_data = {}
    int_results_data = {}

    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    sample_data['SampleName'] = sheet1.cell_value(rowx=25, colx=1)  # B26
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 2 up to row 2 + n
    for row in range(2, 2 + n + 1):
        # Add a column named 'Peak' with value equal to the current loop number n
        int_results_data['Peak'] = [row - 1]
        
        row_data = {}
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)
        # Append the row data to the list
        rows_data.append({**int_results_data, **row_data})

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    return sample_data, int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Loop through each file in the directory
    for filename in os.listdir(parent_directory):
        # Check if the file ends with ".xls"
        if filename.endswith(".xls"):
            file_path = os.path.join(parent_directory, filename)
            
            # Extract data from specific cells
            sample_data, int_results_df = extract_data_from_cells(file_path)
            
            # Append the extracted data to the respective lists
            sample_data_list.append(sample_data)
            int_results_df_list.append(int_results_df)

    # Convert the extracted data into dataframes
    sample_df = pd.DataFrame(sample_data_list)
    int_results_df_combined = pd.concat(int_results_df_list)
    
    # Print the extracted dataframes
    print("Sample Dataframe:")
    print(sample_df)
    print("\nIntResults Dataframe:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "int_results.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()


Enter the parent directory path:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28\2AJ-1401.D


WARNING *** file size (36244) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Sample Dataframe:
  SampleName
0     ladder

IntResults Dataframe:
  Peak   RetTime         Area     Height     Width  Symmetry   Baseline  \
0  [1]  2.962707   455.443756  25.940151  0.254837  0.867126  -4.687346   
1  [2]  3.230931     264.9758  16.417297    0.2263  0.294813  -6.366199   
2  [3]  3.722415  1017.263245  36.858269  0.373992  0.476035  -9.442472   
3  [4]  4.433238   647.048218  34.472748  0.258982  0.930943 -13.891619   
4  [5]  5.141259   215.237579   5.513213   0.46747  5.577894  -5.279715   
5  [6]  5.902357    122.67334   7.320348  0.257508  1.150406   0.083779   
6  [7]                                                                    
7  [8]                                                                    

  TimeStart LevelStart BaselineStart   TimeEnd  
0  2.730381   6.832913     -3.233181  3.166339  
1  3.166339  15.

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\JoelTencer\\Documents\\Agilent HPLC 2\\SEC\\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28\\2AJ-1401.D\\int_results.csv'

In [33]:
import os
import xlrd
import pandas as pd

def extract_data_from_cells(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Initialize dictionaries to store extracted data
    sample_data = {}
    int_results_data = {}

    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    sample_data['SampleName'] = sheet1.cell_value(rowx=25, colx=1)  # B26
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Initialize an empty list to store row data
    rows_data = []
    
    # Loop through cells for each value starting from row 2 up to row 2 + n
    for row in range(2, 2 + n):
        # Add a column named 'Peak' with value equal to the current loop number n
        int_results_data['Peak'] = [row -1]
        
        row_data = {}
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)
        # Append the row data to the list
        rows_data.append({**int_results_data, **row_data})

    # Create DataFrame from the list of row data
    int_results_df = pd.DataFrame(rows_data)

    return sample_data, int_results_df

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_df_list = []

    # Loop through each file in the directory
    for filename in os.listdir(parent_directory):
        # Check if the file ends with ".xls"
        if filename.endswith(".xls"):
            file_path = os.path.join(parent_directory, filename)
            
            # Extract data from specific cells
            sample_data, int_results_df = extract_data_from_cells(file_path)
            
            # Append the extracted data to the respective lists
            sample_data_list.append(sample_data)
            int_results_df_list.append(int_results_df)

    # Convert the extracted data into dataframes
    sample_df = pd.DataFrame(sample_data_list)
    int_results_df_combined = pd.concat(int_results_df_list)
    
    # Print the extracted dataframes
    print("Sample Dataframe:")
    print(sample_df)
    print("\nIntResults Dataframe:")
    print(int_results_df_combined)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "int_results.csv")
    int_results_df_combined.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()


Enter the parent directory path:  import os import xlrd import pandas as pd  def extract_data_from_cells(file_path):     # Open the Excel file     wb = xlrd.open_workbook(file_path)          # Initialize dictionaries to store extracted data     sample_data = {}     int_results_data = {}      # Extract data from Sheet1 (SampleName)     sheet1 = wb.sheet_by_name('Sheet1')     sample_data['SampleName'] = sheet1.cell_value(rowx=25, colx=1)  # B26          # Extract data from IntResults sheet     int_results_sheet = wb.sheet_by_name('IntResults1')          # Check the value of cell B2     n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n          # Initialize an empty list to store row data     rows_data = []          # Loop through cells for each value starting from row 2 up to row 2 + n     for row in range(2, 2 + n):         # Add a column named 'Peak' with value equal to the current loop number n         int_results_data['Peak'] = [row -1]     

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'import os import xlrd import pandas as pd  def extract_data_from_cells(file_path):     # Open the Excel file     wb = xlrd.open_workbook(file_path)          # Initialize dictionaries to store extracted data     sample_data = {}     int_results_data = {}      # Extract data from Sheet1 (SampleName)     sheet1 = wb.sheet_by_name(\'Sheet1\')     sample_data[\'SampleName\'] = sheet1.cell_value(rowx=25, colx=1)  # B26          # Extract data from IntResults sheet     int_results_sheet = wb.sheet_by_name(\'IntResults1\')          # Check the value of cell B2     n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n          # Initialize an empty list to store row data     rows_data = []          # Loop through cells for each value starting from row 2 up to row 2 + n     for row in range(2, 2 + n):         # Add a column named \'Peak\' with value equal to the current loop number 

In [24]:
import os
import xlrd
import pandas as pd

def extract_data_from_cells(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Initialize dictionaries to store extracted data
    sample_data = {}
    int_results_data = {}

    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    sample_data['SampleName'] = sheet1.cell_value(rowx=25, colx=1)  # B26
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Loop through cells for each value starting from row 2 up to row 2 + n
    for row in range(2, 2 + n):
        row_data = {}
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)
        int_results_data[row] = row_data

    return sample_data, int_results_data

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_data_list = []

    # Loop through each file in the directory
    for filename in os.listdir(parent_directory):
        # Check if the file ends with ".xls"
        if filename.endswith(".xls"):
            file_path = os.path.join(parent_directory, filename)
            
            # Extract data from specific cells
            sample_data, int_results_data = extract_data_from_cells(file_path)
            
            # Append the extracted data to the respective lists
            sample_data_list.append(sample_data)
            int_results_data_list.append(int_results_data)

    # Convert the extracted data into dataframes
    sample_df = pd.DataFrame(sample_data_list)
    int_results_df = pd.DataFrame(int_results_data_list)
    
    # Print the extracted dataframes
    print("Sample Dataframe:")
    print(sample_df)
    print("\nIntResults Dataframe:")
    print(int_results_df)
    
    # Save IntResults DataFrame to CSV
    int_results_csv_path = os.path.join(parent_directory, "int_results.csv")
    int_results_df.to_csv(int_results_csv_path, index=False)
    print(f"IntResults DataFrame saved to: {int_results_csv_path}")

if __name__ == "__main__":
    main()


Enter the parent directory path:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28\2AJ-1401.D


WARNING *** file size (36244) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Sample Dataframe:
  SampleName
0     ladder

IntResults Dataframe:
                                                   2  \
0  {'RetTime': 2.962707042694092, 'Area': 455.443...   

                                                   3  \
0  {'RetTime': 3.230930805206299, 'Area': 264.975...   

                                                   4  \
0  {'RetTime': 3.722414970397949, 'Area': 1017.26...   

                                                   5  \
0  {'RetTime': 4.433237552642822, 'Area': 647.048...   

                                                   6  \
0  {'RetTime': 5.14125919342041, 'Area': 215.2375...   

                                                   7  \
0  {'RetTime': 5.90235710144043, 'Area': 122.6733...   

                                                   8  
0  {'RetTime': '', 'Area': '', 'Height': '', 'Wid...  
In

In [18]:
import os
import xlrd
import pandas as pd

def extract_data_from_cells(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Initialize dictionaries to store extracted data
    sample_data = {}
    int_results_data = {}

    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    sample_data['SampleName'] = sheet1.cell_value(rowx=25, colx=1)  # B26
    
    # Extract data from IntResults sheet
    int_results_sheet = wb.sheet_by_name('IntResults1')
    
    # Check the value of cell B2
    n = int(int_results_sheet.cell_value(rowx=1, colx=1))  # Assuming B2 contains the value of n
    
    # Loop through cells for each value starting from row 2 up to row 2 + n
    for row in range(2, 2 + n):
        row_data = {}
        for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
            row_data[label] = int_results_sheet.cell_value(rowx=row, colx=col)
        int_results_data[row] = row_data

    return sample_data, int_results_data

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_data_list = []

    # Loop through each file in the directory
    for filename in os.listdir(parent_directory):
        # Check if the file ends with ".xls"
        if filename.endswith(".xls"):
            file_path = os.path.join(parent_directory, filename)
            
            # Extract data from specific cells
            sample_data, int_results_data = extract_data_from_cells(file_path)
            
            # Append the extracted data to the respective lists
            sample_data_list.append(sample_data)
            int_results_data_list.append(int_results_data)

    # Convert the extracted data into dataframes
    sample_df = pd.DataFrame(sample_data_list)
    int_results_df = pd.DataFrame(int_results_data_list)
    
    # Print the extracted dataframes
    print("Sample Dataframe:")
    print(sample_df)
    print("\nIntResults Dataframe:")
    print(int_results_df)

if __name__ == "__main__":
    main()


Enter the parent directory path:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28\2AJ-1401.D


WARNING *** file size (36244) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Sample Dataframe:
  SampleName
0     ladder

IntResults Dataframe:
                                                   2  \
0  {'RetTime': 2.962707042694092, 'Area': 455.443...   

                                                   3  \
0  {'RetTime': 3.230930805206299, 'Area': 264.975...   

                                                   4  \
0  {'RetTime': 3.722414970397949, 'Area': 1017.26...   

                                                   5  \
0  {'RetTime': 4.433237552642822, 'Area': 647.048...   

                                                   6  \
0  {'RetTime': 5.14125919342041, 'Area': 215.2375...   

                                                   7  \
0  {'RetTime': 5.90235710144043, 'Area': 122.6733...   

                                                   8  
0  {'RetTime': '', 'Area': '', 'Height': '', 'Wid...  


In [22]:
%who

extract_data_from_cells	 main	 merge_csv_files	 openpyxl	 os	 pd	 xlrd	 


In [11]:
# This script works, still gives error:
# WARNING *** file size (31238) not 512 + multiple of sector size (512)
# WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero

import os
import xlrd
import pandas as pd

def extract_data_from_cells(file_path):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Initialize dictionaries to store extracted data
    sample_data = {}
    int_results_data = {}

    # Extract data from Sheet1 (SampleName)
    sheet1 = wb.sheet_by_name('Sheet1')
    sample_data['SampleName'] = sheet1.cell_value(rowx=25, colx=1)  # B26
    
    # Extract data from IntResults1 (RetTime, Area, Height, Width, Symmetry, Baseline, TimeStart, LevelStart, BaselineStart, TimeEnd)
    int_results_sheet = wb.sheet_by_name('IntResults1')
    for col, label in zip(range(4, 14), ['RetTime', 'Area', 'Height', 'Width', 'Symmetry', 'Baseline', 'TimeStart', 'LevelStart', 'BaselineStart', 'TimeEnd']):
        int_results_data[label] = int_results_sheet.cell_value(rowx=1, colx=col)  # Row 2, Column E to N

    return sample_data, int_results_data

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize lists to store extracted data
    sample_data_list = []
    int_results_data_list = []

    # Loop through each file in the directory
    for filename in os.listdir(parent_directory):
        # Check if the file ends with ".xls"
        if filename.endswith(".xls"):
            file_path = os.path.join(parent_directory, filename)
            
            # Extract data from specific cells
            sample_data, int_results_data = extract_data_from_cells(file_path)
            
            # Append the extracted data to the respective lists
            sample_data_list.append(sample_data)
            int_results_data_list.append(int_results_data)

    # Convert the extracted data into dataframes
    sample_df = pd.DataFrame(sample_data_list)
    int_results_df = pd.DataFrame(int_results_data_list)
    
    # Print the extracted dataframes
    print("Sample Dataframe:")
    print(sample_df)
    print("\nIntResults Dataframe:")
    print(int_results_df)

if __name__ == "__main__":
    main()


Enter the parent directory path:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 04 18 770 771 772 773 ELUTIONS 2024-04-18 14-01-28\2AJ-1401.D


WARNING *** file size (36244) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Sample Dataframe:
  SampleName
0     ladder

IntResults Dataframe:
    RetTime        Area    Height     Width  Symmetry  Baseline  TimeStart  \
0  2.629555  106.284401  6.798275  0.228397  1.765334 -2.602099      2.212   

   LevelStart  BaselineStart   TimeEnd  
0         0.0       0.011444  2.730381  


In [4]:


def extract_data_from_cells(file_path, cell_coordinates):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Select the worksheet named 'Sheet1'
    sheet = wb.sheet_by_name('Sheet1')
    
    # Initialize a dictionary to store extracted data
    extracted_data = {}

    for cell_coord in cell_coordinates:
        # Extract row and column indices from the cell coordinates
        column, row = xlrd.sheet.split_cell(cell_coord)
        
        # Get the value of the cell
        value = sheet.cell_value(rowx=row-1, colx=column-1)
        
        # Store the value in the dictionary
        extracted_data[cell_coord] = value

    return extracted_data

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize an empty dataframe to store extracted data
    extracted_df = pd.DataFrame()

    # Loop through each file in the directory
    for filename in os.listdir(parent_directory):
        # Check if the file ends with ".xls"
        if filename.endswith(".xls"):
            file_path = os.path.join(parent_directory, filename)
            
            # Cell coordinates from which you want to extract data
            cell_coordinates = ["A1", "B2", "C3"]  # Example cell coordinates
            
            # Extract data from specific cells
            extracted_data = extract_data_from_cells(file_path, cell_coordinates)
            
            # Convert the extracted data into a dataframe
            df = pd.DataFrame(extracted_data, index=[0])
            
            # Append the dataframe to the extracted_df
            extracted_df = pd.concat([extracted_df, df], ignore_index=True)

    # Print the extracted dataframe
    print("Extracted Dataframe:")
    print(extracted_df)

if __name__ == "__main__":
    main()


Enter the parent directory path:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 04 26 769-773 ELUTIONS POOL PBS 2024-04-26 14-57-25\2BB-0501.D


WARNING *** file size (31238) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


AttributeError: module 'xlrd.sheet' has no attribute 'split_cell'

In [5]:
import os
import xlrd
import pandas as pd

def extract_data_from_cells(file_path, cell_coordinates):
    # Open the Excel file
    wb = xlrd.open_workbook(file_path)
    
    # Select the worksheet named 'Sheet1'
    sheet = wb.sheet_by_name('Sheet1')
    
    # Initialize a dictionary to store extracted data
    extracted_data = {}

    for cell_coord in cell_coordinates:
        # Extract row and column indices from the cell coordinates
        column, row = xlrd.cellname.split(cell_coord)
        
        # Get the value of the cell
        value = sheet.cell_value(rowx=int(row)-1, colx=int(column)-1)
        
        # Store the value in the dictionary
        extracted_data[cell_coord] = value

    return extracted_data

def main():
    # Prompt user to input the parent directory
    parent_directory = input("Enter the parent directory path: ")

    # Initialize an empty dataframe to store extracted data
    extracted_df = pd.DataFrame()

    # Loop through each file in the directory
    for filename in os.listdir(parent_directory):
        # Check if the file ends with ".xls"
        if filename.endswith(".xls"):
            file_path = os.path.join(parent_directory, filename)
            
            # Cell coordinates from which you want to extract data
            cell_coordinates = ["A1", "B2", "C3"]  # Example cell coordinates
            
            # Extract data from specific cells
            extracted_data = extract_data_from_cells(file_path, cell_coordinates)
            
            # Convert the extracted data into a dataframe
            df = pd.DataFrame(extracted_data, index=[0])
            
            # Append the dataframe to the extracted_df
            extracted_df = pd.concat([extracted_df, df], ignore_index=True)

    # Print the extracted dataframe
    print("Extracted Dataframe:")
    print(extracted_df)

if __name__ == "__main__":
    main()


Enter the parent directory path:  C:\Users\JoelTencer\Documents\Agilent HPLC 2\SEC\2024 04 26 769-773 ELUTIONS POOL PBS 2024-04-26 14-57-25\2BB-0501.D


WARNING *** file size (31238) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


AttributeError: 'function' object has no attribute 'split'

In [7]:
def merge_csv_files(parent_folder):
    # Initialize an empty dataframe
    merged_df = pd.DataFrame()

    # Loop through each folder in the parent folder
    for folder_name in os.listdir(parent_folder):
        folder_path = os.path.join(parent_folder, folder_name)
        
        # Check if the current item is a directory
        if os.path.isdir(folder_path):
            # Loop through each file in the directory
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)
                
                # Check if the file is a CSV file
                if file_name.endswith('01.CSV'):
                    # Read the CSV file into a dataframe
                    df = pd.read_csv(file_path, encoding='iso-8859-1')
                    
                    # Append the dataframe to the merged dataframe
                    merged_df = merged_df.append(df, ignore_index=True)

    return merged_df

def main():
    # Specify the parent folder containing subfolders with CSV files
    #parent_folder = "10.2.20.249/Lab Data/Joel Tencer/Agilent HPLC 1/HIC\WX71117_2 2024-04-01 10-57-02"
    parent_folder = "C:/Users/JoelTencer/Documents/HPLC/HIC/WX71117_2 2024-04-01 10-57-02"
    
    # Merge CSV files from subfolders into a single dataframe
    merged_df = merge_csv_files(parent_folder)
    
    # Export the merged dataframe as a single CSV file
    merged_df.to_csv("merged_data.csv", index=False)
    print("Merged data exported successfully.")

if __name__ == "__main__":
    main()

AttributeError: 'DataFrame' object has no attribute 'append'

In [8]:


def extract_data_from_cells(file_path, cell_coordinates):
    # Load the Excel file
    wb = openpyxl.load_workbook(file_path)
    
    # Select the worksheet named 'Sheet1'
    sheet = wb['Sheet1']
    
    # Initialize a dictionary to store extracted data
    extracted_data = {}

    for cell_coord in cell_coordinates:
        # Extract row and column indices from the cell coordinates
        column, row = openpyxl.utils.cell.coordinate_from_string(cell_coord)
        
        # Get the value of the cell
        value = sheet.cell(row=row, column=column).value
        
        # Store the value in the dictionary
        extracted_data[cell_coord] = value

    return extracted_data

def main():
    # File path
    file_path = "example.xlsx"  # Replace with your Excel file path
    
    # Cell coordinates from which you want to extract data
    cell_coordinates = ["A1", "B2", "C3"]  # Example cell coordinates
    
    # Extract data from specific cells
    extracted_data = extract_data_from_cells(file_path, cell_coordinates)
    
    # Print the extracted data
    for cell_coord, value in extracted_data.items():
        print(f"Data extracted from cell {cell_coord}: {value}")

if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'example.xlsx'